# Veolia


In [ ]:
"""
Veolia ETL Transformation Script (Google Colab Version)
Transforms Veolia waste registry data to Urbyn aggregated format.

Uses ETL mapping file DIRECTLY:
- Déchet sheet: Déchets prestataire → Déchet fin
- Paramètres sheet: Déchet fin → Déchets agrégé
- Traitement générique sheet: Treatment code mapping
- Site sheet: Site name mapping
"""

import pandas as pd
import numpy as np
from datetime import datetime
import os

# =============================================================================
# CONFIGURATION - UPDATE THESE PATHS FOR YOUR ENVIRONMENT
# =============================================================================
INPUT_FILES = {
    'Veolia - Toulouse': '/content/Reporting_Veolia - Toulouse_Capegmini_03_25.xlsx',
    'Veolia - Grasse': '/content/Reporting_Veolia - Grasse_Capegmini_03_25.xlsx',
}
TEMPLATE_FILE = '/content/Modèle vierge de Registre des déchets et Reporting des coûts SLIMAN.xlsx'
ETL_FILE = '/content/ETL _ Mapping registre déchets prestataire vers Urbyn.xlsx'
OUTPUT_FILE = '/content/Veolia_Registre_Agrege_OUTPUT.xlsx'

# Prestataire search pattern (partial match in ETL)
PRESTATAIRE_PATTERN = 'Veolia'

TEMPLATE_HEADER_ROW = 8
OUTPUT_SHEET = 'Registre des déchets (Mouvement'

CLIENT = "CAPGEMINI TECHNOLOGY SERVICES"
GROUPE = "Capgemini"
GROUPE_PRESTATAIRE = "Veolia"
TYPE_PRESTATAIRE = "Privé"


class ETLMapper:
    """Loads and applies mappings from the ETL file"""

    def __init__(self, etl_file, prestataire_pattern):
        self.etl_file = etl_file
        self.prestataire_pattern = prestataire_pattern

        print("="*80)
        print(f"LOADING ETL MAPPINGS FOR: {prestataire_pattern}")
        print("="*80)

        self.load_dechet_mapping()
        self.load_dechet_to_agrege_mapping()
        self.load_traitement_mapping()
        self.load_site_mapping()

    def load_dechet_mapping(self):
        """Load Déchet sheet: Déchets prestataire → Déchet fin"""
        df = pd.read_excel(self.etl_file, sheet_name='Déchet')

        # Filter by prestataire pattern
        mask = df['Nom prestataire (FORMULE)'].str.contains(
            self.prestataire_pattern, case=False, na=False
        )
        df_filtered = df[mask]

        print(f"✓ Déchet sheet: {len(df_filtered)} mappings for '{self.prestataire_pattern}'")

        # Build lookup: déchets prestataire → déchet fin
        self.dechet_lookup = {}
        self.default_dechet_fin = None

        for _, row in df_filtered.iterrows():
            prest = row.get('Nom des déchets prestataire')
            urbyn = row.get('Nom des déchets Urbyn')

            if pd.isna(prest):
                # Default mapping for empty/NaN waste names
                if pd.notna(urbyn):
                    self.default_dechet_fin = urbyn
            elif pd.notna(urbyn):
                self.dechet_lookup[str(prest).strip().lower()] = urbyn

        print(f"  - {len(self.dechet_lookup)} explicit mappings")
        print(f"  - Default (for empty): {self.default_dechet_fin}")

    def load_dechet_to_agrege_mapping(self):
        """Load Paramètres sheet: Déchet fin (Name) → Déchets agrégé (Category)"""
        df = pd.read_excel(self.etl_file, sheet_name='Paramètres')

        self.dechet_to_agrege = {}
        for _, row in df[['Category', 'Name']].dropna().iterrows():
            name = str(row['Name']).strip().lower()
            category = str(row['Category']).strip()
            self.dechet_to_agrege[name] = category

        print(f"✓ Paramètres sheet: {len(self.dechet_to_agrege)} Déchet fin → Déchets agrégé mappings")

    def load_traitement_mapping(self):
        """Load Traitement générique sheet"""
        df = pd.read_excel(self.etl_file, sheet_name='Traitement générique')

        self.traitement_lookup = {}
        for _, row in df.iterrows():
            key = str(row.get('Concatener déchet & code de traitement prestataire', '')).strip()
            if key:
                self.traitement_lookup[key] = {
                    'code': row.get('Code traitement retraité'),
                    'traitement': row.get('Traitement')
                }

        print(f"✓ Traitement générique: {len(self.traitement_lookup)} treatment mappings")

    def load_site_mapping(self):
        """Load Site sheet"""
        df = pd.read_excel(self.etl_file, sheet_name='Site')

        mask = df['Nom prestataire (FORMULE)'].str.contains(
            self.prestataire_pattern, case=False, na=False
        )
        df_filtered = df[mask]

        print(f"✓ Site sheet: {len(df_filtered)} sites for '{self.prestataire_pattern}'")

        self.site_lookup = {}
        for _, row in df_filtered.iterrows():
            site_prest = row.get('Nom site prestataire')
            if pd.notna(site_prest):
                self.site_lookup[str(site_prest).strip().lower()] = {
                    'nom_site': row.get('Nom site Urbyn'),
                    'code_prestation': row.get('Code de la prestation'),
                    'prestataire': row.get('Nom prestataire (FORMULE)')
                }

    def map_dechet_prestataire_to_fin(self, dechet_prest):
        """Map Déchets prestataire → Déchet fin"""
        if pd.isna(dechet_prest) or str(dechet_prest).strip() == '':
            return self.default_dechet_fin

        key = str(dechet_prest).strip().lower()
        return self.dechet_lookup.get(key, self.default_dechet_fin)

    def map_dechet_fin_to_agrege(self, dechet_fin):
        """Map Déchet fin → Déchets agrégé"""
        if pd.isna(dechet_fin):
            return None

        key = str(dechet_fin).strip().lower()
        return self.dechet_to_agrege.get(key)

    def map_traitement(self, dechets_agrege, code_prest):
        """Map Déchets agrégé + Code prestataire → Code final + Traitement"""
        if pd.isna(dechets_agrege):
            return None, None

        # Try with code
        if pd.notna(code_prest) and str(code_prest).strip():
            lookup_key = f"{dechets_agrege}{str(code_prest).strip()}"
            if lookup_key in self.traitement_lookup:
                result = self.traitement_lookup[lookup_key]
                return result['code'], result['traitement']

        # Try without code (default for this waste type)
        if dechets_agrege in self.traitement_lookup:
            result = self.traitement_lookup[dechets_agrege]
            return result['code'], result['traitement']

        return None, None

    def map_site(self, site_name):
        """Map site name to Urbyn site info"""
        if pd.isna(site_name):
            return None

        key = str(site_name).strip().lower()

        # Exact match
        if key in self.site_lookup:
            return self.site_lookup[key]

        # Partial match
        for lookup_key, value in self.site_lookup.items():
            if key in lookup_key or lookup_key in key:
                return value

        return None


def read_inputs(input_files):
    """Read all input files"""
    print("\n" + "="*80)
    print("READING INPUT FILES")
    print("="*80)

    dfs = []
    for name, filepath in input_files.items():
        if os.path.exists(filepath):
            df = pd.read_excel(filepath, sheet_name=0)
            df['_source_file'] = name
            print(f"✓ {name}: {len(df)} rows")
            dfs.append(df)
        else:
            print(f"⚠ File not found: {filepath}")

    if not dfs:
        raise ValueError("No input files found!")

    return pd.concat(dfs, ignore_index=True)


def read_template(template_file):
    """Read template for column structure"""
    print("\n" + "="*80)
    print("READING TEMPLATE")
    print("="*80)

    df = pd.read_excel(template_file, sheet_name=0, header=TEMPLATE_HEADER_ROW)
    print(f"✓ Template: {len(df.columns)} columns")
    return df


def transform(df_input, df_template, mapper):
    """Transform input data to output format"""
    print("\n" + "="*80)
    print("TRANSFORMING DATA")
    print("="*80)

    output_rows = []

    for idx, row in df_input.iterrows():
        # === SITE MAPPING ===
        site_name = row.get('Lieu de collecte', '')
        site_info = mapper.map_site(site_name)

        # === WASTE MAPPING ===
        dechets_prest = row.get('Matière', '')
        dechet_fin = mapper.map_dechet_prestataire_to_fin(dechets_prest)
        dechets_agrege = mapper.map_dechet_fin_to_agrege(dechet_fin)

        # === TREATMENT MAPPING ===
        code_prest = row.get('Code de traitement', '')
        code_final, traitement = mapper.map_traitement(dechets_agrege, code_prest)

        # === WEIGHT CONVERSION (tonnes → kg) ===
        poids = row.get('Poids', 0)
        if pd.isna(poids):
            poids = 0
        unite = str(row.get('Unité poids', '')).lower()
        masse_kg = float(poids) * 1000 if 'tonne' in unite else float(poids)

        # === BUILD OUTPUT ROW ===
        new_row = {
            'Libellé': None,
            'Groupe': GROUPE,
            'Code site': site_info['nom_site'].split(' - ')[0] if site_info and pd.notna(site_info.get('nom_site')) and ' - ' in str(site_info.get('nom_site', '')) else None,
            'Nom du site': site_info['nom_site'] if site_info else 'Site inconnu',
            'Nom du client': CLIENT,
            'Type de porteur': 'FM',
            'Commentaire mouvement': None,
            'Code de la prestation': site_info['code_prestation'] if site_info else None,
            'Prestataire': site_info['prestataire'] if site_info else row.get('_source_file', GROUPE_PRESTATAIRE),
            'Groupe de Prestataire': GROUPE_PRESTATAIRE,
            'Type de prestataire': TYPE_PRESTATAIRE,
            'Périodicité': 'Jour',
            'Date début registre': row.get('Date de réalisation'),
            'Date fin registre': row.get('Date de réalisation'),
            'Code déchet prestataire': row.get('COD CED'),
            'Déchet fin': dechet_fin,
            'Déchets agrégé': dechets_agrege,
            'Déchets prestataire': dechets_prest if pd.notna(dechets_prest) else dechet_fin,
            'Masse totale (kg)': masse_kg,
            'Nombre de contenants': row.get('Quantité facturée', 1),
            'Volume contenant (L)': None,
            'Type de contenant': row.get('Matériel'),
            'Volume total (L)': None,
            'Nature de quantités collectées': 'Masse',
            'Qualité quantités': 'Document prestataire',
            'Précision estimations des quantités': None,
            'Traitement': traitement,
            'Traitement prestataire': None,
            'Code traitement': code_final if code_final else code_prest,
            'Code traitement prestataire': code_prest,
            'Qualité du Traitement': 'Document prestataire',
            "N° de BSD/BSDD": row.get("N° de bon d'enlèvement"),
            'N° de recépissé': row.get('Récépissé du transporteur'),
            'Transporteur': row.get('Nom du transporteur'),
            'Transporteur prestataire': row.get('Nom du transporteur'),
            "Plaque d'immatriculation": None,
            'Exutoire intermédiaire': None,
            'Exutoire intermédiaire prestataire': None,
            "Qualité de l'exutoire intermédiaire": None,
            'Exutoire final': row.get('Exutoire'),
            'Exutoire final prestataire': row.get('Exutoire'),
            "Qualité de l'exutoire final": 'Document prestataire',
            'Période de clôture': None,
            'Statut du mouvement': 'Réalisée',
            'Commentaire': row.get('Votre commentaire'),
        }

        output_rows.append(new_row)

    # Create DataFrame and align with template columns
    df_output = pd.DataFrame(output_rows)

    for col in df_template.columns:
        if col not in df_output.columns:
            df_output[col] = None

    df_output = df_output[df_template.columns]

    print(f"✓ Transformed {len(df_output)} rows")
    return df_output


def save_output(df_output, output_file):
    """Save output file"""
    print("\n" + "="*80)
    print("SAVING OUTPUT")
    print("="*80)

    df_output.to_excel(output_file, sheet_name=OUTPUT_SHEET, index=False)
    print(f"✓ Saved: {output_file}")


def main():
    print("\n" + "="*80)
    print("VEOLIA ETL TRANSFORMATION")
    print("="*80)

    try:
        mapper = ETLMapper(ETL_FILE, PRESTATAIRE_PATTERN)
        df_input = read_inputs(INPUT_FILES)
        df_template = read_template(TEMPLATE_FILE)
        df_output = transform(df_input, df_template, mapper)
        save_output(df_output, OUTPUT_FILE)

        print("\n" + "="*80)
        print("✓✓✓ SUCCESS ✓✓✓")
        print("="*80)

    except Exception as e:
        print(f"\n✗ ERROR: {e}")
        import traceback
        traceback.print_exc()


if __name__ == "__main__":
    main()


VEOLIA ETL TRANSFORMATION
LOADING ETL MAPPINGS FOR: Veolia
✓ Déchet sheet: 103 mappings for 'Veolia'
  - 25 explicit mappings
  - Default (for empty): None
✓ Paramètres sheet: 121 Déchet fin → Déchets agrégé mappings
✓ Traitement générique: 169 treatment mappings
✓ Site sheet: 97 sites for 'Veolia'

READING INPUT FILES
✓ Veolia - Toulouse: 1 rows
✓ Veolia - Grasse: 9 rows

READING TEMPLATE
✓ Template: 64 columns

TRANSFORMING DATA
✓ Transformed 10 rows

SAVING OUTPUT
✓ Saved: /content/Veolia_Registre_Agrege_OUTPUT.xlsx

✓✓✓ SUCCESS ✓✓✓


# Suez

In [ ]:
"""
Suez ETL Transformation Script (Google Colab Version)
Transforms Suez waste registry data to Urbyn aggregated format.

Uses ETL mapping file DIRECTLY:
- Déchet sheet: Déchets prestataire → Déchet fin
- Paramètres sheet: Déchet fin → Déchets agrégé
- Traitement générique sheet: Treatment code mapping
- Site sheet: Site name mapping
"""

import pandas as pd
import numpy as np
from datetime import datetime
import os

# =============================================================================
# CONFIGURATION - UPDATE THESE PATHS FOR YOUR ENVIRONMENT
# =============================================================================
INPUT_FILES = {
    'Suez - Midi Pyrénées': '/content/Reporting_Suez - Midi Pyrénées_Capgemini_03_25.xlsx',
    'Suez - Montpellier': '/content/Reporting_Suez - Montpellier_Capgemini_03_25.xlsx',
}
TEMPLATE_FILE = '/content/Modèle vierge de Registre des déchets et Reporting des coûts SLIMAN.xlsx'
ETL_FILE = '/content/ETL _ Mapping registre déchets prestataire vers Urbyn.xlsx'
OUTPUT_FILE = '/content/Suez_Registre_Agrege_OUTPUT.xlsx'

# Prestataire search pattern (partial match in ETL)
PRESTATAIRE_PATTERN = 'Suez'

TEMPLATE_HEADER_ROW = 8
INPUT_HEADER_ROW = 6  # Suez has sub-headers
OUTPUT_SHEET = 'Registre des déchets (Mouvement'

CLIENT = "CAPGEMINI TECHNOLOGY SERVICES"
GROUPE = "Capgemini"
GROUPE_PRESTATAIRE = "Suez"
TYPE_PRESTATAIRE = "Privé"


class ETLMapper:
    """Loads and applies mappings from the ETL file"""

    def __init__(self, etl_file, prestataire_pattern):
        self.etl_file = etl_file
        self.prestataire_pattern = prestataire_pattern

        print("="*80)
        print(f"LOADING ETL MAPPINGS FOR: {prestataire_pattern}")
        print("="*80)

        self.load_dechet_mapping()
        self.load_dechet_to_agrege_mapping()
        self.load_traitement_mapping()
        self.load_site_mapping()

    def load_dechet_mapping(self):
        """Load Déchet sheet: Déchets prestataire → Déchet fin"""
        df = pd.read_excel(self.etl_file, sheet_name='Déchet')

        mask = df['Nom prestataire (FORMULE)'].str.contains(
            self.prestataire_pattern, case=False, na=False
        )
        df_filtered = df[mask]

        print(f"✓ Déchet sheet: {len(df_filtered)} mappings for '{self.prestataire_pattern}'")

        self.dechet_lookup = {}
        self.default_dechet_fin = None

        for _, row in df_filtered.iterrows():
            prest = row.get('Nom des déchets prestataire')
            urbyn = row.get('Nom des déchets Urbyn')

            if pd.isna(prest):
                if pd.notna(urbyn):
                    self.default_dechet_fin = urbyn
            elif pd.notna(urbyn):
                self.dechet_lookup[str(prest).strip().lower()] = urbyn

        print(f"  - {len(self.dechet_lookup)} explicit mappings")
        print(f"  - Default (for empty): {self.default_dechet_fin}")

    def load_dechet_to_agrege_mapping(self):
        """Load Paramètres sheet: Déchet fin (Name) → Déchets agrégé (Category)"""
        df = pd.read_excel(self.etl_file, sheet_name='Paramètres')

        self.dechet_to_agrege = {}
        for _, row in df[['Category', 'Name']].dropna().iterrows():
            name = str(row['Name']).strip().lower()
            category = str(row['Category']).strip()
            self.dechet_to_agrege[name] = category

        print(f"✓ Paramètres sheet: {len(self.dechet_to_agrege)} Déchet fin → Déchets agrégé mappings")

    def load_traitement_mapping(self):
        """Load Traitement générique sheet"""
        df = pd.read_excel(self.etl_file, sheet_name='Traitement générique')

        self.traitement_lookup = {}
        for _, row in df.iterrows():
            key = str(row.get('Concatener déchet & code de traitement prestataire', '')).strip()
            if key:
                self.traitement_lookup[key] = {
                    'code': row.get('Code traitement retraité'),
                    'traitement': row.get('Traitement')
                }

        print(f"✓ Traitement générique: {len(self.traitement_lookup)} treatment mappings")

    def load_site_mapping(self):
        """Load Site sheet"""
        df = pd.read_excel(self.etl_file, sheet_name='Site')

        mask = df['Nom prestataire (FORMULE)'].str.contains(
            self.prestataire_pattern, case=False, na=False
        )
        df_filtered = df[mask]

        print(f"✓ Site sheet: {len(df_filtered)} sites for '{self.prestataire_pattern}'")

        self.site_lookup = {}
        for _, row in df_filtered.iterrows():
            site_prest = row.get('Nom site prestataire')
            if pd.notna(site_prest):
                self.site_lookup[str(site_prest).strip().lower()] = {
                    'nom_site': row.get('Nom site Urbyn'),
                    'code_prestation': row.get('Code de la prestation'),
                    'prestataire': row.get('Nom prestataire (FORMULE)')
                }

    def map_dechet_prestataire_to_fin(self, dechet_prest):
        """Map Déchets prestataire → Déchet fin"""
        if pd.isna(dechet_prest) or str(dechet_prest).strip() == '':
            return self.default_dechet_fin

        key = str(dechet_prest).strip().lower()
        return self.dechet_lookup.get(key, self.default_dechet_fin)

    def map_dechet_fin_to_agrege(self, dechet_fin):
        """Map Déchet fin → Déchets agrégé"""
        if pd.isna(dechet_fin):
            return None

        key = str(dechet_fin).strip().lower()
        return self.dechet_to_agrege.get(key)

    def map_traitement(self, dechets_agrege, code_prest):
        """Map Déchets agrégé + Code prestataire → Code final + Traitement"""
        if pd.isna(dechets_agrege):
            return None, None

        if pd.notna(code_prest) and str(code_prest).strip():
            lookup_key = f"{dechets_agrege}{str(code_prest).strip()}"
            if lookup_key in self.traitement_lookup:
                result = self.traitement_lookup[lookup_key]
                return result['code'], result['traitement']

        if dechets_agrege in self.traitement_lookup:
            result = self.traitement_lookup[dechets_agrege]
            return result['code'], result['traitement']

        return None, None

    def map_site(self, site_name):
        """Map site name to Urbyn site info"""
        if pd.isna(site_name):
            return None

        key = str(site_name).strip().lower()

        if key in self.site_lookup:
            return self.site_lookup[key]

        for lookup_key, value in self.site_lookup.items():
            if key in lookup_key or lookup_key in key:
                return value

        return None


def read_inputs(input_files):
    """Read all Suez input files"""
    print("\n" + "="*80)
    print("READING INPUT FILES")
    print("="*80)

    dfs = []
    for name, filepath in input_files.items():
        if os.path.exists(filepath):
            # Suez files have header at row 6
            df = pd.read_excel(filepath, sheet_name='Registre déchets', header=INPUT_HEADER_ROW)
            # Filter out empty rows
            df = df[df['Type'].notna()].reset_index(drop=True)
            df['_source_file'] = name
            print(f"✓ {name}: {len(df)} rows")
            dfs.append(df)
        else:
            print(f"⚠ File not found: {filepath}")

    if not dfs:
        raise ValueError("No input files found!")

    return pd.concat(dfs, ignore_index=True)


def read_template(template_file):
    """Read template for column structure"""
    print("\n" + "="*80)
    print("READING TEMPLATE")
    print("="*80)

    df = pd.read_excel(template_file, sheet_name=0, header=TEMPLATE_HEADER_ROW)
    print(f"✓ Template: {len(df.columns)} columns")
    return df


def transform(df_input, df_template, mapper):
    """Transform input data to output format"""
    print("\n" + "="*80)
    print("TRANSFORMING DATA")
    print("="*80)

    output_rows = []

    for idx, row in df_input.iterrows():
        # === SITE MAPPING ===
        site_name = row.get('Nom du site', '')
        site_info = mapper.map_site(site_name)

        # === WASTE MAPPING ===
        dechets_prest = row.get('Matière', '')
        dechet_fin = mapper.map_dechet_prestataire_to_fin(dechets_prest)
        dechets_agrege = mapper.map_dechet_fin_to_agrege(dechet_fin)

        # === TREATMENT MAPPING ===
        code_prest = row.get('Code R/D', '')
        code_final, traitement = mapper.map_traitement(dechets_agrege, code_prest)

        # === WEIGHT CONVERSION (Suez uses tonnes) ===
        poids = row.get('Qté pesée (dont estimée)', 0)
        if pd.isna(poids):
            poids = 0
        masse_kg = float(poids) * 1000  # Suez is always in tonnes

        # === BUILD OUTPUT ROW ===
        new_row = {
            'Libellé': None,
            'Groupe': GROUPE,
            'Code site': site_info['nom_site'].split(' - ')[0] if site_info and pd.notna(site_info.get('nom_site')) and ' - ' in str(site_info.get('nom_site', '')) else None,
            'Nom du site': site_info['nom_site'] if site_info else 'Site inconnu',
            'Nom du client': CLIENT,
            'Type de porteur': 'FM',
            'Commentaire mouvement': None,
            'Code de la prestation': site_info['code_prestation'] if site_info else None,
            'Prestataire': site_info['prestataire'] if site_info else row.get('_source_file', GROUPE_PRESTATAIRE),
            'Groupe de Prestataire': GROUPE_PRESTATAIRE,
            'Type de prestataire': TYPE_PRESTATAIRE,
            'Périodicité': 'Jour',
            'Date début registre': row.get("Date de l'expédition"),
            'Date fin registre': row.get("Date de l'expédition"),
            'Code déchet prestataire': row.get('Code déchet (CED)'),
            'Déchet fin': dechet_fin,
            'Déchets agrégé': dechets_agrege,
            'Déchets prestataire': dechets_prest if pd.notna(dechets_prest) else dechet_fin,
            'Masse totale (kg)': masse_kg,
            'Nombre de contenants': row.get('Qté de matériel collectée', 1),
            'Volume contenant (L)': row.get('Volume du matériel'),
            'Type de contenant': None,
            'Volume total (L)': None,
            'Nature de quantités collectées': 'Masse',
            'Qualité quantités': 'Document prestataire',
            'Précision estimations des quantités': None,
            'Traitement': traitement,
            'Traitement prestataire': row.get('Libellé R/D'),
            'Code traitement': code_final if code_final else code_prest,
            'Code traitement prestataire': code_prest,
            'Qualité du Traitement': 'Document prestataire',
            'N° de BSD/BSDD': row.get('N° du BSD'),
            'N° de recépissé': row.get('N° récépissé'),
            'Transporteur': row.get('Nom.1'),
            'Transporteur prestataire': row.get('Nom.1'),
            "Plaque d'immatriculation": None,
            'Exutoire intermédiaire': None,
            'Exutoire intermédiaire prestataire': None,
            "Qualité de l'exutoire intermédiaire": None,
            'Exutoire final': row.get("Nom de l'installation"),
            'Exutoire final prestataire': row.get("Nom de l'installation"),
            "Qualité de l'exutoire final": 'Document prestataire',
            'Période de clôture': None,
            'Statut du mouvement': 'Réalisée',
            'Commentaire': None,
        }

        output_rows.append(new_row)

    df_output = pd.DataFrame(output_rows)

    for col in df_template.columns:
        if col not in df_output.columns:
            df_output[col] = None

    df_output = df_output[df_template.columns]

    print(f"✓ Transformed {len(df_output)} rows")
    return df_output


def save_output(df_output, output_file):
    """Save output file"""
    print("\n" + "="*80)
    print("SAVING OUTPUT")
    print("="*80)

    df_output.to_excel(output_file, sheet_name=OUTPUT_SHEET, index=False)
    print(f"✓ Saved: {output_file}")


def main():
    print("\n" + "="*80)
    print("SUEZ ETL TRANSFORMATION")
    print("="*80)

    try:
        mapper = ETLMapper(ETL_FILE, PRESTATAIRE_PATTERN)
        df_input = read_inputs(INPUT_FILES)
        df_template = read_template(TEMPLATE_FILE)
        df_output = transform(df_input, df_template, mapper)
        save_output(df_output, OUTPUT_FILE)

        print("\n" + "="*80)
        print("✓✓✓ SUCCESS ✓✓✓")
        print("="*80)

    except Exception as e:
        print(f"\n✗ ERROR: {e}")
        import traceback
        traceback.print_exc()


if __name__ == "__main__":
    main()


SUEZ ETL TRANSFORMATION
LOADING ETL MAPPINGS FOR: Suez
✓ Déchet sheet: 34 mappings for 'Suez'
  - 22 explicit mappings
  - Default (for empty): None
✓ Paramètres sheet: 121 Déchet fin → Déchets agrégé mappings
✓ Traitement générique: 169 treatment mappings
✓ Site sheet: 9 sites for 'Suez'

READING INPUT FILES
✓ Suez - Midi Pyrénées: 94 rows
✓ Suez - Montpellier: 9 rows

READING TEMPLATE
✓ Template: 64 columns

TRANSFORMING DATA
✓ Transformed 103 rows

SAVING OUTPUT
✓ Saved: /content/Suez_Registre_Agrege_OUTPUT.xlsx

✓✓✓ SUCCESS ✓✓✓


# Apeyron

In [ ]:
"""
Apeyron ETL Transformation Script (Google Colab Version)
Transforms Apeyron waste registry data to Urbyn aggregated format.

Uses ETL mapping file DIRECTLY:
- Déchet sheet: Déchets prestataire → Déchet fin
- Paramètres sheet: Déchet fin → Déchets agrégé
- Traitement générique sheet: Treatment code mapping
- Site sheet: Site name mapping
"""

import pandas as pd
import numpy as np
from datetime import datetime
import os
import re

# =============================================================================
# CONFIGURATION - UPDATE THESE PATHS FOR YOUR ENVIRONMENT
# =============================================================================
INPUT_FILE = '/content/Reporting_Apeyron_Capgemini_03_25.xlsx'
TEMPLATE_FILE = '/content/Modèle vierge de Registre des déchets et Reporting des coûts SLIMAN.xlsx'
ETL_FILE = '/content/ETL _ Mapping registre déchets prestataire vers Urbyn.xlsx'
OUTPUT_FILE = '/content/Apeyron_Registre_Agrege_OUTPUT.xlsx'

# Prestataire search pattern (partial match in ETL)
PRESTATAIRE_PATTERN = 'Apeyron'

TEMPLATE_HEADER_ROW = 8
INPUT_HEADER_ROW = 1  # Apeyron header row
OUTPUT_SHEET = 'Registre des déchets (Mouvement'

CLIENT = "CAPGEMINI TECHNOLOGY SERVICES"
GROUPE = "Capgemini"
GROUPE_PRESTATAIRE = "Apeyron Environnement"
TYPE_PRESTATAIRE = "Privé"


class ETLMapper:
    """Loads and applies mappings from the ETL file"""

    def __init__(self, etl_file, prestataire_pattern):
        self.etl_file = etl_file
        self.prestataire_pattern = prestataire_pattern

        print("="*80)
        print(f"LOADING ETL MAPPINGS FOR: {prestataire_pattern}")
        print("="*80)

        self.load_dechet_mapping()
        self.load_dechet_to_agrege_mapping()
        self.load_traitement_mapping()
        self.load_site_mapping()

    def load_dechet_mapping(self):
        """Load Déchet sheet: Déchets prestataire → Déchet fin"""
        df = pd.read_excel(self.etl_file, sheet_name='Déchet')

        mask = df['Nom prestataire (FORMULE)'].str.contains(
            self.prestataire_pattern, case=False, na=False
        )
        df_filtered = df[mask]

        print(f"✓ Déchet sheet: {len(df_filtered)} mappings for '{self.prestataire_pattern}'")

        self.dechet_lookup = {}
        self.default_dechet_fin = None

        for _, row in df_filtered.iterrows():
            prest = row.get('Nom des déchets prestataire')
            urbyn = row.get('Nom des déchets Urbyn')

            if pd.isna(prest):
                if pd.notna(urbyn):
                    self.default_dechet_fin = urbyn
            elif pd.notna(urbyn):
                self.dechet_lookup[str(prest).strip().lower()] = urbyn

        print(f"  - {len(self.dechet_lookup)} explicit mappings")
        print(f"  - Default (for empty): {self.default_dechet_fin}")

    def load_dechet_to_agrege_mapping(self):
        """Load Paramètres sheet: Déchet fin (Name) → Déchets agrégé (Category)"""
        df = pd.read_excel(self.etl_file, sheet_name='Paramètres')

        self.dechet_to_agrege = {}
        for _, row in df[['Category', 'Name']].dropna().iterrows():
            name = str(row['Name']).strip().lower()
            category = str(row['Category']).strip()
            self.dechet_to_agrege[name] = category

        print(f"✓ Paramètres sheet: {len(self.dechet_to_agrege)} Déchet fin → Déchets agrégé mappings")

    def load_traitement_mapping(self):
        """Load Traitement générique sheet"""
        df = pd.read_excel(self.etl_file, sheet_name='Traitement générique')

        self.traitement_lookup = {}
        for _, row in df.iterrows():
            key = str(row.get('Concatener déchet & code de traitement prestataire', '')).strip()
            if key:
                self.traitement_lookup[key] = {
                    'code': row.get('Code traitement retraité'),
                    'traitement': row.get('Traitement')
                }

        print(f"✓ Traitement générique: {len(self.traitement_lookup)} treatment mappings")

    def load_site_mapping(self):
        """Load Site sheet"""
        df = pd.read_excel(self.etl_file, sheet_name='Site')

        mask = df['Nom prestataire (FORMULE)'].str.contains(
            self.prestataire_pattern, case=False, na=False
        )
        df_filtered = df[mask]

        print(f"✓ Site sheet: {len(df_filtered)} sites for '{self.prestataire_pattern}'")

        self.site_lookup = {}
        for _, row in df_filtered.iterrows():
            site_prest = row.get('Nom site prestataire')
            if pd.notna(site_prest):
                self.site_lookup[str(site_prest).strip().lower()] = {
                    'nom_site': row.get('Nom site Urbyn'),
                    'code_prestation': row.get('Code de la prestation'),
                    'prestataire': row.get('Nom prestataire (FORMULE)')
                }

    def map_dechet_prestataire_to_fin(self, dechet_prest):
        """Map Déchets prestataire → Déchet fin"""
        if pd.isna(dechet_prest) or str(dechet_prest).strip() == '':
            return self.default_dechet_fin

        key = str(dechet_prest).strip().lower()
        return self.dechet_lookup.get(key, self.default_dechet_fin)

    def map_dechet_fin_to_agrege(self, dechet_fin):
        """Map Déchet fin → Déchets agrégé"""
        if pd.isna(dechet_fin):
            return None

        key = str(dechet_fin).strip().lower()
        return self.dechet_to_agrege.get(key)

    def map_traitement(self, dechets_agrege, code_prest):
        """Map Déchets agrégé + Code prestataire → Code final + Traitement"""
        if pd.isna(dechets_agrege):
            return None, None

        if pd.notna(code_prest) and str(code_prest).strip():
            lookup_key = f"{dechets_agrege}{str(code_prest).strip()}"
            if lookup_key in self.traitement_lookup:
                result = self.traitement_lookup[lookup_key]
                return result['code'], result['traitement']

        if dechets_agrege in self.traitement_lookup:
            result = self.traitement_lookup[dechets_agrege]
            return result['code'], result['traitement']

        return None, None

    def map_site(self, site_name):
        """Map site name to Urbyn site info"""
        if pd.isna(site_name):
            return None

        key = str(site_name).strip().lower()

        if key in self.site_lookup:
            return self.site_lookup[key]

        for lookup_key, value in self.site_lookup.items():
            if key in lookup_key or lookup_key in key:
                return value

        return None

    def get_volume_from_container(self, container_code):
        """Extract volume in liters from container code (e.g., BAC035 → 35)"""
        if pd.isna(container_code):
            return None

        container_str = str(container_code).upper()
        match = re.search(r'BAC\s*0*(\d+)', container_str)
        if match:
            return int(match.group(1))

        return None

    def get_container_type_name(self, volume_liters):
        """Get standardized container type name from volume"""
        if volume_liters is None:
            return None

        names = {
            35: 'Bac 2 roues - 35 L',
            120: 'Bac 2 roues - 120 L',
            240: 'Bac 2 roues - 240 L',
            660: 'Bac 4 roues - 660 L',
            770: 'Bac 4 roues - 770 L',
            1100: 'Bac 4 roues - 1100 L',
        }
        return names.get(volume_liters, f'Bac - {volume_liters} L')


def read_input(input_file):
    """Read Apeyron input file"""
    print("\n" + "="*80)
    print("READING INPUT FILE")
    print("="*80)

    df = pd.read_excel(input_file, sheet_name=0, header=INPUT_HEADER_ROW)
    df = df[df['Date'].notna()].reset_index(drop=True)

    print(f"✓ Loaded: {len(df)} rows")
    return df


def read_template(template_file):
    """Read template for column structure"""
    print("\n" + "="*80)
    print("READING TEMPLATE")
    print("="*80)

    df = pd.read_excel(template_file, sheet_name=0, header=TEMPLATE_HEADER_ROW)
    print(f"✓ Template: {len(df.columns)} columns")
    return df


def transform(df_input, df_template, mapper):
    """Transform input data to output format"""
    print("\n" + "="*80)
    print("TRANSFORMING DATA")
    print("="*80)

    output_rows = []

    for idx, row in df_input.iterrows():
        # === SITE MAPPING ===
        site_name = row.get('Nom Etablissement', '')
        site_info = mapper.map_site(site_name)

        if site_info is None:
            # Skip rows without site mapping
            continue

        # === WASTE MAPPING (Apeyron usually has no waste column) ===
        dechets_prest = None
        for col in ['Type de matière', 'Matière', 'Type déchet', 'Déchet']:
            if col in row.index and pd.notna(row.get(col)):
                dechets_prest = row.get(col)
                break

        dechet_fin = mapper.map_dechet_prestataire_to_fin(dechets_prest)
        dechets_agrege = mapper.map_dechet_fin_to_agrege(dechet_fin)

        # === TREATMENT MAPPING ===
        code_prest = None
        for col in ['Code traitement', 'Traitement', 'Code R/D']:
            if col in row.index and pd.notna(row.get(col)):
                code_prest = row.get(col)
                break

        code_final, traitement = mapper.map_traitement(dechets_agrege, code_prest)

        # === WEIGHT (Apeyron already in kg) ===
        masse_kg = row.get('Poids Total Kg', 0)
        if pd.isna(masse_kg):
            masse_kg = 0

        # === CONTAINER ===
        nb_contenants = row.get('Nb Bac', 1)
        if pd.isna(nb_contenants):
            nb_contenants = 1

        container_code = row.get('Volume Bac', '')
        volume_contenant = mapper.get_volume_from_container(container_code)
        type_contenant = mapper.get_container_type_name(volume_contenant)
        volume_total = volume_contenant * nb_contenants if volume_contenant else None

        # === BUILD OUTPUT ROW ===
        new_row = {
            'Libellé': None,
            'Groupe': GROUPE,
            'Code site': site_info['nom_site'].split(' - ')[0] if site_info and pd.notna(site_info.get('nom_site')) and ' - ' in str(site_info.get('nom_site', '')) else None,
            'Nom du site': site_info['nom_site'] if site_info else 'Site inconnu',
            'Nom du client': CLIENT,
            'Type de porteur': 'FM',
            'Commentaire mouvement': None,
            'Code de la prestation': site_info['code_prestation'] if site_info else None,
            'Prestataire': GROUPE_PRESTATAIRE,
            'Groupe de Prestataire': GROUPE_PRESTATAIRE,
            'Type de prestataire': TYPE_PRESTATAIRE,
            'Périodicité': 'Jour',
            'Date début registre': row.get('Date'),
            'Date fin registre': row.get('Date'),
            'Code déchet prestataire': '20_01_08',
            'Déchet fin': dechet_fin,
            'Déchets agrégé': dechets_agrege,
            'Déchets prestataire': dechets_prest if pd.notna(dechets_prest) else dechet_fin,
            'Masse totale (kg)': masse_kg,
            'Nombre de contenants': nb_contenants,
            'Volume contenant (L)': volume_contenant,
            'Type de contenant': type_contenant,
            'Volume total (L)': volume_total,
            'Nature de quantités collectées': 'Volume et Masse',
            'Qualité quantités': 'Document prestataire',
            'Précision estimations des quantités': None,
            'Traitement': traitement,
            'Traitement prestataire': None,
            'Code traitement': code_final if code_final else code_prest,
            'Code traitement prestataire': code_prest,
            'Qualité du Traitement': 'Document prestataire',
            'N° de BSD/BSDD': None,
            'N° de recépissé': None,
            'Transporteur': None,
            'Transporteur prestataire': GROUPE_PRESTATAIRE,
            "Plaque d'immatriculation": None,
            'Exutoire intermédiaire': None,
            'Exutoire intermédiaire prestataire': 'Exutoire intermédiaire inconnu',
            "Qualité de l'exutoire intermédiaire": 'Inconnu',
            'Exutoire final': None,
            'Exutoire final prestataire': None,
            "Qualité de l'exutoire final": 'Document prestataire',
            'Période de clôture': None,
            'Statut du mouvement': 'Réalisée',
            'Commentaire': None,
        }

        output_rows.append(new_row)

    df_output = pd.DataFrame(output_rows)

    for col in df_template.columns:
        if col not in df_output.columns:
            df_output[col] = None

    df_output = df_output[df_template.columns]

    print(f"✓ Transformed {len(df_output)} rows")
    return df_output


def save_output(df_output, output_file):
    """Save output file"""
    print("\n" + "="*80)
    print("SAVING OUTPUT")
    print("="*80)

    df_output.to_excel(output_file, sheet_name=OUTPUT_SHEET, index=False)
    print(f"✓ Saved: {output_file}")


def main():
    print("\n" + "="*80)
    print("APEYRON ETL TRANSFORMATION")
    print("="*80)

    try:
        mapper = ETLMapper(ETL_FILE, PRESTATAIRE_PATTERN)
        df_input = read_input(INPUT_FILE)
        df_template = read_template(TEMPLATE_FILE)
        df_output = transform(df_input, df_template, mapper)
        save_output(df_output, OUTPUT_FILE)

        print("\n" + "="*80)
        print("✓✓✓ SUCCESS ✓✓✓")
        print("="*80)

    except Exception as e:
        print(f"\n✗ ERROR: {e}")
        import traceback
        traceback.print_exc()


if __name__ == "__main__":
    main()


APEYRON ETL TRANSFORMATION
LOADING ETL MAPPINGS FOR: Apeyron
✓ Déchet sheet: 1 mappings for 'Apeyron'
  - 0 explicit mappings
  - Default (for empty): Biodéchets
✓ Paramètres sheet: 121 Déchet fin → Déchets agrégé mappings
✓ Traitement générique: 169 treatment mappings
✓ Site sheet: 1 sites for 'Apeyron'

READING INPUT FILE
✓ Loaded: 101 rows

READING TEMPLATE
✓ Template: 64 columns

TRANSFORMING DATA
✓ Transformed 22 rows

SAVING OUTPUT
✓ Saved: /content/Apeyron_Registre_Agrege_OUTPUT.xlsx

✓✓✓ SUCCESS ✓✓✓


# les alchimistes

In [ ]:
"""
Les Alchimistes ETL Transformation Script (Google Colab Version)
Transforms Les Alchimistes waste registry data to Urbyn aggregated format.

Uses ETL mapping file DIRECTLY:
- Déchet sheet: Déchets prestataire → Déchet fin
- Paramètres sheet: Déchet fin → Déchets agrégé
- Traitement générique sheet: Treatment code mapping
- Site sheet: Site name mapping
"""

import pandas as pd
import numpy as np
from datetime import datetime
import os
import re

# =============================================================================
# CONFIGURATION - UPDATE THESE PATHS FOR YOUR ENVIRONMENT
# =============================================================================
INPUT_FILES = {
    'Lesalchimistes - Idf': '/content/Reporting_Lesalchimistes - Idf_Capgemini_03_25.xlsx',
    'Lesalchimistes - Lille': '/content/Reporting_Lesalchimistes - Lille_Capgemini_03_25.xlsx',
    'Lesalchimistes - Languedoc - Montpellier': '/content/Reporting_Lesalchimistes - Languedoc - Montpellier_Capgemini_03_25.xlsx',
}
TEMPLATE_FILE = '/content/Modèle vierge de Registre des déchets et Reporting des coûts SLIMAN.xlsx'
ETL_FILE = '/content/ETL _ Mapping registre déchets prestataire vers Urbyn.xlsx'
OUTPUT_FILE = '/content/LesAlchimistes_Registre_Agrege_OUTPUT.xlsx'

# Prestataire search pattern (partial match in ETL)
PRESTATAIRE_PATTERN = 'alchimiste'

TEMPLATE_HEADER_ROW = 8
OUTPUT_SHEET = 'Registre des déchets (Mouvement'

CLIENT = "CAPGEMINI TECHNOLOGY SERVICES"
GROUPE = "Capgemini"
GROUPE_PRESTATAIRE = "Les Alchimistes"
TYPE_PRESTATAIRE = "Privé"


class ETLMapper:
    """Loads and applies mappings from the ETL file"""

    def __init__(self, etl_file, prestataire_pattern):
        self.etl_file = etl_file
        self.prestataire_pattern = prestataire_pattern

        print("="*80)
        print(f"LOADING ETL MAPPINGS FOR: {prestataire_pattern}")
        print("="*80)

        self.load_dechet_mapping()
        self.load_dechet_to_agrege_mapping()
        self.load_traitement_mapping()
        self.load_site_mapping()

    def load_dechet_mapping(self):
        """Load Déchet sheet: Déchets prestataire → Déchet fin"""
        df = pd.read_excel(self.etl_file, sheet_name='Déchet')

        mask = df['Nom prestataire (FORMULE)'].str.contains(
            self.prestataire_pattern, case=False, na=False
        )
        df_filtered = df[mask]

        print(f"✓ Déchet sheet: {len(df_filtered)} mappings for '{self.prestataire_pattern}'")

        self.dechet_lookup = {}
        self.default_dechet_fin = None

        for _, row in df_filtered.iterrows():
            prest = row.get('Nom des déchets prestataire')
            urbyn = row.get('Nom des déchets Urbyn')

            if pd.isna(prest):
                if pd.notna(urbyn):
                    self.default_dechet_fin = urbyn
            elif pd.notna(urbyn):
                self.dechet_lookup[str(prest).strip().lower()] = urbyn

        print(f"  - {len(self.dechet_lookup)} explicit mappings")
        print(f"  - Default (for empty): {self.default_dechet_fin}")

    def load_dechet_to_agrege_mapping(self):
        """Load Paramètres sheet: Déchet fin (Name) → Déchets agrégé (Category)"""
        df = pd.read_excel(self.etl_file, sheet_name='Paramètres')

        self.dechet_to_agrege = {}
        for _, row in df[['Category', 'Name']].dropna().iterrows():
            name = str(row['Name']).strip().lower()
            category = str(row['Category']).strip()
            self.dechet_to_agrege[name] = category

        print(f"✓ Paramètres sheet: {len(self.dechet_to_agrege)} Déchet fin → Déchets agrégé mappings")

    def load_traitement_mapping(self):
        """Load Traitement générique sheet"""
        df = pd.read_excel(self.etl_file, sheet_name='Traitement générique')

        self.traitement_lookup = {}
        for _, row in df.iterrows():
            key = str(row.get('Concatener déchet & code de traitement prestataire', '')).strip()
            if key:
                self.traitement_lookup[key] = {
                    'code': row.get('Code traitement retraité'),
                    'traitement': row.get('Traitement')
                }

        print(f"✓ Traitement générique: {len(self.traitement_lookup)} treatment mappings")

    def load_site_mapping(self):
        """Load Site sheet"""
        df = pd.read_excel(self.etl_file, sheet_name='Site')

        mask = df['Nom prestataire (FORMULE)'].str.contains(
            self.prestataire_pattern, case=False, na=False
        )
        df_filtered = df[mask]

        print(f"✓ Site sheet: {len(df_filtered)} sites for '{self.prestataire_pattern}'")

        self.site_lookup = {}
        for _, row in df_filtered.iterrows():
            site_prest = row.get('Nom site prestataire')
            if pd.notna(site_prest):
                self.site_lookup[str(site_prest).strip().lower()] = {
                    'nom_site': row.get('Nom site Urbyn'),
                    'code_prestation': row.get('Code de la prestation'),
                    'prestataire': row.get('Nom prestataire (FORMULE)')
                }

    def map_dechet_prestataire_to_fin(self, dechet_prest):
        """Map Déchets prestataire → Déchet fin"""
        if pd.isna(dechet_prest) or str(dechet_prest).strip() == '':
            return self.default_dechet_fin

        key = str(dechet_prest).strip().lower()
        return self.dechet_lookup.get(key, self.default_dechet_fin)

    def map_dechet_fin_to_agrege(self, dechet_fin):
        """Map Déchet fin → Déchets agrégé"""
        if pd.isna(dechet_fin):
            return None

        key = str(dechet_fin).strip().lower()
        return self.dechet_to_agrege.get(key)

    def map_traitement(self, dechets_agrege, code_prest):
        """Map Déchets agrégé + Code prestataire → Code final + Traitement"""
        if pd.isna(dechets_agrege):
            return None, None

        if pd.notna(code_prest) and str(code_prest).strip():
            lookup_key = f"{dechets_agrege}{str(code_prest).strip()}"
            if lookup_key in self.traitement_lookup:
                result = self.traitement_lookup[lookup_key]
                return result['code'], result['traitement']

        if dechets_agrege in self.traitement_lookup:
            result = self.traitement_lookup[dechets_agrege]
            return result['code'], result['traitement']

        return None, None

    def map_site(self, site_name):
        """Map site name to Urbyn site info"""
        if pd.isna(site_name):
            return None

        key = str(site_name).strip().lower()

        if key in self.site_lookup:
            return self.site_lookup[key]

        for lookup_key, value in self.site_lookup.items():
            if key in lookup_key or lookup_key in key:
                return value

        return None


def detect_header_row(filepath):
    """Detect where the actual header row is in Les Alchimistes files"""
    df_raw = pd.read_excel(filepath, sheet_name=0, header=None, nrows=20)

    for i, row in df_raw.iterrows():
        row_values = [str(v).lower() for v in row.values if pd.notna(v)]
        if any('date' in v for v in row_values) and any('poids' in v or 'kg' in v for v in row_values):
            return i

    return 0


def read_inputs(input_files):
    """Read all Les Alchimistes input files"""
    print("\n" + "="*80)
    print("READING INPUT FILES")
    print("="*80)

    dfs = []
    for name, filepath in input_files.items():
        if os.path.exists(filepath):
            header_row = detect_header_row(filepath)
            df = pd.read_excel(filepath, sheet_name=0, header=header_row)

            # Find date column and filter
            date_col = None
            for col in df.columns:
                if 'date' in str(col).lower():
                    date_col = col
                    break

            if date_col:
                df = df[df[date_col].notna()].reset_index(drop=True)

            df['_source_file'] = name
            print(f"✓ {name}: {len(df)} rows (header at row {header_row})")
            dfs.append(df)
        else:
            print(f"⚠ File not found: {filepath}")

    if not dfs:
        raise ValueError("No input files found!")

    return pd.concat(dfs, ignore_index=True)


def read_template(template_file):
    """Read template for column structure"""
    print("\n" + "="*80)
    print("READING TEMPLATE")
    print("="*80)

    df = pd.read_excel(template_file, sheet_name=0, header=TEMPLATE_HEADER_ROW)
    print(f"✓ Template: {len(df.columns)} columns")
    return df


def find_column(df, possible_names):
    """Find column by trying multiple possible names"""
    for name in possible_names:
        for col in df.columns:
            if name.lower() in str(col).lower():
                return col
    return None


def transform(df_input, df_template, mapper):
    """Transform input data to output format"""
    print("\n" + "="*80)
    print("TRANSFORMING DATA")
    print("="*80)

    # Find columns dynamically
    date_col = find_column(df_input, ['date'])
    site_col = find_column(df_input, ['site', 'client', 'établissement', 'etablissement'])
    weight_col = find_column(df_input, ['poids', 'kg', 'masse', 'quantité'])
    waste_col = find_column(df_input, ['matière', 'déchet', 'type'])
    code_col = find_column(df_input, ['code', 'traitement', 'r/d'])

    print(f"  Date column: {date_col}")
    print(f"  Site column: {site_col}")
    print(f"  Weight column: {weight_col}")
    print(f"  Waste column: {waste_col}")

    output_rows = []

    for idx, row in df_input.iterrows():
        # === SITE MAPPING ===
        site_name = row.get(site_col, '') if site_col else ''
        site_info = mapper.map_site(site_name)

        if site_info is None:
            # Skip rows without site mapping
            continue

        # === WASTE MAPPING ===
        dechets_prest = row.get(waste_col, '') if waste_col else ''
        dechet_fin = mapper.map_dechet_prestataire_to_fin(dechets_prest)
        dechets_agrege = mapper.map_dechet_fin_to_agrege(dechet_fin)

        # === TREATMENT MAPPING ===
        code_prest = row.get(code_col, '') if code_col else ''
        code_final, traitement = mapper.map_traitement(dechets_agrege, code_prest)

        # === WEIGHT ===
        masse_kg = row.get(weight_col, 0) if weight_col else 0
        if pd.isna(masse_kg):
            masse_kg = 0

        # Get prestataire from site mapping or source file
        prestataire = site_info.get('prestataire', row.get('_source_file', GROUPE_PRESTATAIRE))

        # === BUILD OUTPUT ROW ===
        new_row = {
            'Libellé': None,
            'Groupe': GROUPE,
            'Code site': site_info['nom_site'].split(' - ')[0] if site_info and pd.notna(site_info.get('nom_site')) and ' - ' in str(site_info.get('nom_site', '')) else None,
            'Nom du site': site_info['nom_site'] if site_info else 'Site inconnu',
            'Nom du client': CLIENT,
            'Type de porteur': 'FM',
            'Commentaire mouvement': None,
            'Code de la prestation': site_info['code_prestation'] if site_info else None,
            'Prestataire': prestataire,
            'Groupe de Prestataire': GROUPE_PRESTATAIRE,
            'Type de prestataire': TYPE_PRESTATAIRE,
            'Périodicité': 'Jour',
            'Date début registre': row.get(date_col) if date_col else None,
            'Date fin registre': row.get(date_col) if date_col else None,
            'Code déchet prestataire': '20_01_08',
            'Déchet fin': dechet_fin,
            'Déchets agrégé': dechets_agrege,
            'Déchets prestataire': dechets_prest if pd.notna(dechets_prest) and str(dechets_prest).strip() else dechet_fin,
            'Masse totale (kg)': masse_kg,
            'Nombre de contenants': 1,
            'Volume contenant (L)': None,
            'Type de contenant': None,
            'Volume total (L)': None,
            'Nature de quantités collectées': 'Masse',
            'Qualité quantités': 'Document prestataire',
            'Précision estimations des quantités': None,
            'Traitement': traitement,
            'Traitement prestataire': None,
            'Code traitement': code_final if code_final else code_prest,
            'Code traitement prestataire': code_prest if pd.notna(code_prest) else code_final,
            'Qualité du Traitement': 'Document prestataire',
            'N° de BSD/BSDD': None,
            'N° de recépissé': None,
            'Transporteur': None,
            'Transporteur prestataire': prestataire,
            "Plaque d'immatriculation": None,
            'Exutoire intermédiaire': None,
            'Exutoire intermédiaire prestataire': None,
            "Qualité de l'exutoire intermédiaire": None,
            'Exutoire final': None,
            'Exutoire final prestataire': None,
            "Qualité de l'exutoire final": 'Document prestataire',
            'Période de clôture': None,
            'Statut du mouvement': 'Réalisée',
            'Commentaire': None,
        }

        output_rows.append(new_row)

    df_output = pd.DataFrame(output_rows)

    for col in df_template.columns:
        if col not in df_output.columns:
            df_output[col] = None

    df_output = df_output[df_template.columns]

    print(f"✓ Transformed {len(df_output)} rows")
    return df_output


def save_output(df_output, output_file):
    """Save output file"""
    print("\n" + "="*80)
    print("SAVING OUTPUT")
    print("="*80)

    df_output.to_excel(output_file, sheet_name=OUTPUT_SHEET, index=False)
    print(f"✓ Saved: {output_file}")


def main():
    print("\n" + "="*80)
    print("LES ALCHIMISTES ETL TRANSFORMATION")
    print("="*80)

    try:
        mapper = ETLMapper(ETL_FILE, PRESTATAIRE_PATTERN)
        df_input = read_inputs(INPUT_FILES)
        df_template = read_template(TEMPLATE_FILE)
        df_output = transform(df_input, df_template, mapper)
        save_output(df_output, OUTPUT_FILE)

        print("\n" + "="*80)
        print("✓✓✓ SUCCESS ✓✓✓")
        print("="*80)

    except Exception as e:
        print(f"\n✗ ERROR: {e}")
        import traceback
        traceback.print_exc()


if __name__ == "__main__":
    main()


LES ALCHIMISTES ETL TRANSFORMATION
LOADING ETL MAPPINGS FOR: alchimiste
✓ Déchet sheet: 28 mappings for 'alchimiste'
  - 4 explicit mappings
  - Default (for empty): Biodéchets
✓ Paramètres sheet: 121 Déchet fin → Déchets agrégé mappings
✓ Traitement générique: 169 treatment mappings
✓ Site sheet: 91 sites for 'alchimiste'

READING INPUT FILES
✓ Lesalchimistes - Idf: 76 rows (header at row 0)
✓ Lesalchimistes - Lille: 5 rows (header at row 0)
✓ Lesalchimistes - Languedoc - Montpellier: 4 rows (header at row 0)

READING TEMPLATE
✓ Template: 64 columns

TRANSFORMING DATA
  Date column: Date
  Site column: Nom du client
  Weight column: Poids net (contenu déchets)
  Waste column: Type de matière
✓ Transformed 9 rows

SAVING OUTPUT
✓ Saved: /content/LesAlchimistes_Registre_Agrege_OUTPUT.xlsx

✓✓✓ SUCCESS ✓✓✓


#Paprec


In [ ]:
"""
Paprec ETL Transformation Script (Google Colab Version)
Transforms Paprec waste registry data to Urbyn aggregated format.

Key differences from other prestataires:
- Site identified by 'Numéro Contrat' (not address)
- Waste type from 'Libellé Qualité'
- Container type from 'Libellé matériel'
- No treatment code in input (derived from ETL based on waste type)
- Weight in 'Poids' (kg)
"""

import pandas as pd
import numpy as np
import os
import re

# =============================================================================
# CONFIGURATION
# =============================================================================
INPUT_FILE = '/content/TABLE_17-12-2025_16H49.xlsx'
TEMPLATE_FILE = '/content/Modèle vierge de Registre des déchets et Reporting des coûts SLIMAN.xlsx'
ETL_FILE = '/content/ETL _ Mapping registre déchets prestataire vers Urbyn.xlsx'
OUTPUT_FILE = '/content/Paprec_Registre_Agrege_OUTPUT.xlsx'

PRESTATAIRE_PATTERN = 'Paprec'
TEMPLATE_HEADER_ROW = 8
OUTPUT_SHEET = 'Registre des déchets (Mouvement'

CLIENT = "CAPGEMINI TECHNOLOGY SERVICES"
GROUPE = "CAPGEMINI TECHNOLOGY SERVICES"
GROUPE_PRESTATAIRE = "Paprec"
TYPE_PRESTATAIRE = "Privé"

# Column patterns for dynamic detection
COLUMN_PATTERNS = {
    'site_id': ['numéro contrat', 'contrat', 'numero contrat', 'n° contrat'],
    'site_ref': ['référence', 'reference', 'adresse'],
    'date': ['date prestation', 'date', 'date collecte'],
    'waste': ['libellé qualité', 'qualité', 'matière', 'déchet'],
    'container': ['libellé matériel', 'matériel', 'contenant', 'bac'],
    'quantity': ['quantité', 'nombre', 'nb'],
    'weight': ['poids', 'masse', 'kg'],
    'bsd': ['numerobe', 'n° be', 'bon enlèvement', 'bsd'],
    'month': ['mois'],
    'year': ['année', 'annee'],
}

# Container volume mapping
CONTAINER_VOLUMES = {
    'bac roulant 660l': 660,
    'bac roulant 340l': 340,
    'bac roulant 770l': 770,
    'bac roulant 240l': 240,
    'bac roulant 120l': 120,
    'caisse palette 600l': 600,
    'palette': 1000,
    'balle': 1000,
    'box': 1000,
    'sac': 110,
    'vrac': None,
    "croqu' feuilles": None,
}

# Container type names
CONTAINER_TYPES = {
    'bac roulant 660l': 'Bac 4 roues - 660 L',
    'bac roulant 340l': 'Bac 2 roues - 340 L',
    'bac roulant 770l': 'Bac 4 roues - 770 L',
    'bac roulant 240l': 'Bac 2 roues - 240 L',
    'bac roulant 120l': 'Bac 2 roues - 120 L',
    'caisse palette 600l': 'Caisse Palette - 600 L',
    'palette': 'Palette - 1 m3',
    'balle': 'Balle - 1 m3',
    'box': 'Box - 1 m3',
    'sac': 'Sac - 110 L',
    'vrac': 'Equipement inconnu',
    "croqu' feuilles": 'Equipement inconnu',
}

# =============================================================================
# UTILITY FUNCTIONS
# =============================================================================
def find_column(df, possible_names):
    for name in possible_names:
        for col in df.columns:
            if name.lower() in str(col).lower():
                return col
    return None

def safe_get(row, column, default=None):
    if column is None:
        return default
    try:
        value = row.get(column)
        return default if pd.isna(value) else value
    except:
        return default

def extract_code_site(nom_site):
    if pd.isna(nom_site):
        return None
    nom_str = str(nom_site)
    return nom_str.split(' - ')[0] if ' - ' in nom_str else None

def get_container_volume(container_name):
    if pd.isna(container_name):
        return None
    key = str(container_name).lower().strip()
    return CONTAINER_VOLUMES.get(key)

def get_container_type(container_name):
    if pd.isna(container_name):
        return 'Equipement inconnu'
    key = str(container_name).lower().strip()
    return CONTAINER_TYPES.get(key, 'Equipement inconnu')

def get_periode_cloture(row, cols):
    """Build période de clôture from month and year"""
    mois = safe_get(row, cols.get('month'))
    annee = safe_get(row, cols.get('year'))
    if mois and annee:
        return f"{int(mois)}_{int(annee)}"
    return None

# =============================================================================
# ETL MAPPER CLASS
# =============================================================================
class ETLMapper:
    def __init__(self, etl_file, prestataire_pattern):
        self.etl_file = etl_file
        self.prestataire_pattern = prestataire_pattern

        print("="*80)
        print(f"LOADING ETL MAPPINGS FOR: {prestataire_pattern}")
        print("="*80)

        self.load_dechet_mapping()
        self.load_dechet_to_agrege_mapping()
        self.load_traitement_mapping()
        self.load_site_mapping()

    def load_dechet_mapping(self):
        df = pd.read_excel(self.etl_file, sheet_name='Déchet')
        mask = df['Nom prestataire (FORMULE)'].str.contains(self.prestataire_pattern, case=False, na=False)
        df_filtered = df[mask]
        print(f"✓ Déchet: {len(df_filtered)} mappings")

        self.dechet_lookup = {}
        self.default_dechet_fin = None
        for _, row in df_filtered.iterrows():
            prest = row.get('Nom des déchets prestataire')
            urbyn = row.get('Nom des déchets Urbyn')
            if pd.isna(prest):
                if pd.notna(urbyn): self.default_dechet_fin = urbyn
            elif pd.notna(urbyn):
                self.dechet_lookup[str(prest).strip().lower()] = urbyn

    def load_dechet_to_agrege_mapping(self):
        df = pd.read_excel(self.etl_file, sheet_name='Paramètres')
        self.dechet_to_agrege = {}
        for _, row in df[['Category', 'Name']].dropna().iterrows():
            self.dechet_to_agrege[str(row['Name']).strip().lower()] = str(row['Category']).strip()
        print(f"✓ Paramètres: {len(self.dechet_to_agrege)} mappings")

    def load_traitement_mapping(self):
        df = pd.read_excel(self.etl_file, sheet_name='Traitement générique')
        self.traitement_lookup = {}
        for _, row in df.iterrows():
            key = str(row.get('Concatener déchet & code de traitement prestataire', '')).strip()
            if key:
                self.traitement_lookup[key] = {
                    'code': row.get('Code traitement retraité'),
                    'traitement': row.get('Traitement')
                }
        print(f"✓ Traitement: {len(self.traitement_lookup)} mappings")

    def load_site_mapping(self):
        df = pd.read_excel(self.etl_file, sheet_name='Site')
        mask = df['Nom prestataire (FORMULE)'].str.contains(self.prestataire_pattern, case=False, na=False)
        df_filtered = df[mask]
        print(f"✓ Site: {len(df_filtered)} mappings")

        self.site_lookup = {}
        for _, row in df_filtered.iterrows():
            site_prest = row.get('Nom site prestataire')
            if pd.notna(site_prest):
                # Store both string and numeric versions of the key
                key = str(site_prest).strip().lower()
                self.site_lookup[key] = {
                    'nom_site': row.get('Nom site Urbyn'),
                    'code_prestation': row.get('Code de la prestation'),
                    'prestataire': row.get('Nom prestataire (FORMULE)')
                }

    def map_dechet(self, dechet_prest):
        if pd.isna(dechet_prest) or str(dechet_prest).strip() == '':
            return self.default_dechet_fin
        key = str(dechet_prest).strip().lower()
        return self.dechet_lookup.get(key, self.default_dechet_fin)

    def map_agrege(self, dechet_fin):
        if pd.isna(dechet_fin): return None
        return self.dechet_to_agrege.get(str(dechet_fin).strip().lower())

    def map_traitement(self, agrege, code=None):
        """Map treatment - Paprec often has no code, so use default for waste type"""
        if pd.isna(agrege): return None, None

        # Try with code if provided
        if pd.notna(code) and str(code).strip():
            key = f"{agrege}{str(code).strip()}"
            if key in self.traitement_lookup:
                r = self.traitement_lookup[key]
                return r['code'], r['traitement']

        # Try default (no code)
        if agrege in self.traitement_lookup:
            r = self.traitement_lookup[agrege]
            return r['code'], r['traitement']

        return None, None

    def map_site(self, site_id):
        """Map site by Numéro Contrat"""
        if pd.isna(site_id): return None

        # Try as string
        key = str(site_id).strip().lower()
        if key in self.site_lookup:
            return self.site_lookup[key]

        # Try as int (in case of numeric ID)
        try:
            key_int = str(int(float(site_id))).lower()
            if key_int in self.site_lookup:
                return self.site_lookup[key_int]
        except:
            pass

        return None

# =============================================================================
# MAIN FUNCTIONS
# =============================================================================
def read_input(input_file):
    print("\n" + "="*80)
    print("READING INPUT FILE")
    print("="*80)

    if not os.path.exists(input_file):
        raise ValueError(f"File not found: {input_file}")

    df = pd.read_excel(input_file, sheet_name=0, header=0)
    print(f"✓ Loaded: {len(df)} rows, {len(df.columns)} columns")

    # Detect columns
    cols = {k: find_column(df, v) for k, v in COLUMN_PATTERNS.items()}
    print("\n  Detected columns:")
    for k, v in cols.items():
        print(f"    {k}: {v if v else 'NOT FOUND'}")

    return df, cols

def read_template(template_file):
    print("\n" + "="*80)
    print("READING TEMPLATE")
    print("="*80)
    df = pd.read_excel(template_file, sheet_name=0, header=TEMPLATE_HEADER_ROW)
    print(f"✓ Template: {len(df.columns)} columns")
    return df

def transform(df_input, df_template, mapper, cols):
    print("\n" + "="*80)
    print("TRANSFORMING DATA")
    print("="*80)

    output_rows = []
    skipped = 0

    for _, row in df_input.iterrows():
        # Site mapping (by Numéro Contrat)
        site_id = safe_get(row, cols.get('site_id'), '')
        site_info = mapper.map_site(site_id)

        if site_info is None:
            skipped += 1
            continue

        # Waste mapping (Libellé Qualité → Déchet fin → Déchets agrégé)
        dechets_prest = safe_get(row, cols.get('waste'), '')
        dechet_fin = mapper.map_dechet(dechets_prest)
        dechets_agrege = mapper.map_agrege(dechet_fin)

        # Treatment mapping (no code in input, derive from waste type)
        code_final, traitement = mapper.map_traitement(dechets_agrege, None)

        # Weight (already in kg)
        masse_kg = safe_get(row, cols.get('weight'), 0)
        if masse_kg: masse_kg = float(masse_kg)

        # Container info
        container_name = safe_get(row, cols.get('container'), '')
        nb_contenants = safe_get(row, cols.get('quantity'), 0)
        volume_contenant = get_container_volume(container_name)
        type_contenant = get_container_type(container_name)
        volume_total = volume_contenant * nb_contenants if volume_contenant and nb_contenants else 0

        # Date
        date_val = safe_get(row, cols.get('date'))

        # Période de clôture
        periode = get_periode_cloture(row, cols)

        # BSD number
        bsd = safe_get(row, cols.get('bsd'))

        new_row = {
            'Libellé': None,
            'Groupe': GROUPE,
            'Code site': extract_code_site(site_info['nom_site']) if site_info else None,
            'Nom du site': site_info['nom_site'] if site_info else 'Site inconnu',
            'Nom du client': CLIENT,
            'Type de porteur': 'FM',
            'Commentaire mouvement': None,
            'Code de la prestation': site_info['code_prestation'] if site_info else None,
            'Prestataire': site_info['prestataire'] if site_info else GROUPE_PRESTATAIRE,
            'Groupe de Prestataire': GROUPE_PRESTATAIRE,
            'Type de prestataire': TYPE_PRESTATAIRE,
            'Périodicité': 'Jour',
            'Date début registre': date_val,
            'Date fin registre': date_val,
            'Code déchet prestataire': None,
            'Déchet fin': dechet_fin,
            'Déchets agrégé': dechets_agrege,
            'Déchets prestataire': dechets_prest if dechets_prest else dechet_fin,
            'Masse totale (kg)': masse_kg,
            'Nombre de contenants': nb_contenants,
            'Volume contenant (L)': volume_contenant,
            'Type de contenant': type_contenant,
            'Volume total (L)': volume_total,
            'Nature de quantités collectées': 'Volume et Masse',
            'Qualité quantités': 'Document prestataire',
            'Précision estimations des quantités': None,
            'Traitement': traitement,
            'Traitement prestataire': None,
            'Code traitement': code_final,
            'Code traitement prestataire': code_final,
            'Qualité du Traitement': 'Document prestataire',
            'N° de BSD/BSDD': bsd,
            'N° de recépissé': None,
            'Transporteur': 'Transporteur inconnu',
            'Transporteur prestataire': None,
            "Plaque d'immatriculation": None,
            'Exutoire intermédiaire': 'Exutoire intermédiaire inconnu',
            'Exutoire intermédiaire prestataire': None,
            "Qualité de l'exutoire intermédiaire": 'Inconnu',
            'Exutoire final': 'Exutoire final inconnu',
            'Exutoire final prestataire': None,
            "Qualité de l'exutoire final": 'Inconnu',
            'Période de clôture': periode,
            'Statut du mouvement': 'Réalisée',
            'Commentaire': None,
        }
        output_rows.append(new_row)

    df_output = pd.DataFrame(output_rows)
    for col in df_template.columns:
        if col not in df_output.columns:
            df_output[col] = None
    df_output = df_output[df_template.columns]

    print(f"✓ Transformed {len(df_output)} rows")
    if skipped:
        print(f"  ⚠ Skipped {skipped} rows (no site mapping)")
    return df_output

def save_output(df_output, output_file):
    print("\n" + "="*80)
    print("SAVING OUTPUT")
    print("="*80)
    df_output.to_excel(output_file, sheet_name=OUTPUT_SHEET, index=False)
    print(f"✓ Saved: {output_file}")

def main():
    print("\n" + "="*80)
    print("PAPREC ETL TRANSFORMATION")
    print("="*80)

    try:
        mapper = ETLMapper(ETL_FILE, PRESTATAIRE_PATTERN)
        df_input, cols = read_input(INPUT_FILE)
        df_template = read_template(TEMPLATE_FILE)
        df_output = transform(df_input, df_template, mapper, cols)
        save_output(df_output, OUTPUT_FILE)

        print("\n" + "="*80)
        print("✓✓✓ SUCCESS ✓✓✓")
        print("="*80)
    except Exception as e:
        print(f"\n✗ ERROR: {e}")
        import traceback
        traceback.print_exc()

if __name__ == "__main__":
    main()


PAPREC ETL TRANSFORMATION
LOADING ETL MAPPINGS FOR: Paprec
✓ Déchet: 365 mappings
✓ Paramètres: 121 mappings
✓ Traitement: 169 mappings
✓ Site: 86 mappings

READING INPUT FILE
✓ Loaded: 127 rows, 10 columns

  Detected columns:
    site_id: Numéro Contrat
    site_ref: Référence
    date: Date Prestation
    waste: Libellé Qualité
    container: Libellé matériel
    quantity: Quantité
    weight: Poids
    bsd: NumeroBE
    month: Mois
    year: Année

READING TEMPLATE
✓ Template: 64 columns

TRANSFORMING DATA
✓ Transformed 119 rows
  ⚠ Skipped 8 rows (no site mapping)

SAVING OUTPUT
✓ Saved: /content/Paprec_Registre_Agrege_OUTPUT.xlsx

✓✓✓ SUCCESS ✓✓✓


#Elise

In [ ]:
"""
Elise ETL Transformation Script (Google Colab Version)
Transforms Elise waste registry data (CSV format) to Urbyn aggregated format.

Key characteristics:
- Input is CSV with semicolon separator
- Site identified by 'NOM' column (site name)
- Waste type from 'Gisement' column
- Weight in 'Quantité (kg)' - already in kg
- Treatment code from 'Code D/R' (intermediate) and 'Code D/R.1' (final)
- Exutoire from 'Installation' column
"""

import pandas as pd
import numpy as np
import os

# =============================================================================
# CONFIGURATION
# =============================================================================
INPUT_FILE = '/content/Reporting_Elise_Capgemini_02_25.csv'
TEMPLATE_FILE = '/content/Modèle vierge de Registre des déchets et Reporting des coûts SLIMAN.xlsx'
ETL_FILE = '/content/ETL _ Mapping registre déchets prestataire vers Urbyn.xlsx'
OUTPUT_FILE = '/content/Elise_Registre_Agrege_OUTPUT.xlsx'

PRESTATAIRE_PATTERN = 'Elise'
TEMPLATE_HEADER_ROW = 8
OUTPUT_SHEET = 'Registre des déchets (Mouvement'

CLIENT = "CAPGEMINI TECHNOLOGY SERVICES"
GROUPE = "Capgemini"
GROUPE_PRESTATAIRE = "Elise"
TYPE_PRESTATAIRE = "Privé"

# Column patterns for dynamic detection
COLUMN_PATTERNS = {
    'site_name': ['nom', 'site', 'client'],
    'site_address': ['n° + rue', 'rue', 'adresse'],
    'site_cp': ['cp'],
    'site_ville': ['ville'],
    'date': ['date collecte', 'date'],
    'waste': ['gisement', 'matière', 'déchet', 'qualité'],
    'waste_code': ['code nomenclature', 'nomenclature', 'code déchet'],
    'weight': ['quantité', 'kg', 'poids', 'masse'],
    'collector_site': ['site'],  # Collector site name
    'treatment_code_intermediate': ['code d/r'],
    'treatment_code_final': ['code d/r.1'],
    'exutoire': ['installation', 'exutoire'],
}

# =============================================================================
# UTILITY FUNCTIONS
# =============================================================================
def find_column(df, possible_names, exclude_cols=None):
    """Find column by keyword, optionally excluding already-found columns"""
    exclude_cols = exclude_cols or []
    for name in possible_names:
        for col in df.columns:
            if col in exclude_cols:
                continue
            if name.lower() in str(col).lower():
                return col
    return None

def safe_get(row, column, default=None):
    if column is None:
        return default
    try:
        value = row.get(column)
        return default if pd.isna(value) else value
    except:
        return default

def extract_code_site(nom_site):
    if pd.isna(nom_site):
        return None
    nom_str = str(nom_site)
    return nom_str.split(' - ')[0] if ' - ' in nom_str else None

def detect_csv_separator(filepath):
    """Detect CSV separator by reading first line"""
    with open(filepath, 'r', encoding='utf-8-sig') as f:
        first_line = f.readline()

    # Count potential separators
    separators = {';': first_line.count(';'), ',': first_line.count(','), '\t': first_line.count('\t')}
    return max(separators, key=separators.get)

def detect_header_row(filepath, sep):
    """Detect header row in CSV"""
    df_raw = pd.read_csv(filepath, sep=sep, header=None, nrows=10, encoding='utf-8-sig')
    for i, row in df_raw.iterrows():
        row_text = ' '.join([str(v).lower() for v in row.values if pd.notna(v)])
        if 'date' in row_text and ('gisement' in row_text or 'quantité' in row_text or 'nom' in row_text):
            return i
    return 0

# =============================================================================
# ETL MAPPER CLASS
# =============================================================================
class ETLMapper:
    def __init__(self, etl_file, prestataire_pattern):
        self.etl_file = etl_file
        self.prestataire_pattern = prestataire_pattern

        print("="*80)
        print(f"LOADING ETL MAPPINGS FOR: {prestataire_pattern}")
        print("="*80)

        self.load_dechet_mapping()
        self.load_dechet_to_agrege_mapping()
        self.load_traitement_mapping()
        self.load_site_mapping()

    def load_dechet_mapping(self):
        df = pd.read_excel(self.etl_file, sheet_name='Déchet')
        mask = df['Nom prestataire (FORMULE)'].str.contains(self.prestataire_pattern, case=False, na=False)
        df_filtered = df[mask]
        print(f"✓ Déchet: {len(df_filtered)} mappings")

        self.dechet_lookup = {}
        self.default_dechet_fin = None
        for _, row in df_filtered.iterrows():
            prest = row.get('Nom des déchets prestataire')
            urbyn = row.get('Nom des déchets Urbyn')
            if pd.isna(prest):
                if pd.notna(urbyn): self.default_dechet_fin = urbyn
            elif pd.notna(urbyn):
                self.dechet_lookup[str(prest).strip().lower()] = urbyn

    def load_dechet_to_agrege_mapping(self):
        df = pd.read_excel(self.etl_file, sheet_name='Paramètres')
        self.dechet_to_agrege = {}
        for _, row in df[['Category', 'Name']].dropna().iterrows():
            self.dechet_to_agrege[str(row['Name']).strip().lower()] = str(row['Category']).strip()
        print(f"✓ Paramètres: {len(self.dechet_to_agrege)} mappings")

    def load_traitement_mapping(self):
        df = pd.read_excel(self.etl_file, sheet_name='Traitement générique')
        self.traitement_lookup = {}
        for _, row in df.iterrows():
            key = str(row.get('Concatener déchet & code de traitement prestataire', '')).strip()
            if key:
                self.traitement_lookup[key] = {
                    'code': row.get('Code traitement retraité'),
                    'traitement': row.get('Traitement')
                }
        print(f"✓ Traitement: {len(self.traitement_lookup)} mappings")

    def load_site_mapping(self):
        df = pd.read_excel(self.etl_file, sheet_name='Site')
        mask = df['Nom prestataire (FORMULE)'].str.contains(self.prestataire_pattern, case=False, na=False)
        df_filtered = df[mask]
        print(f"✓ Site: {len(df_filtered)} mappings")

        self.site_lookup = {}
        for _, row in df_filtered.iterrows():
            site_prest = row.get('Nom site prestataire')
            if pd.notna(site_prest):
                key = str(site_prest).strip().lower()
                self.site_lookup[key] = {
                    'nom_site': row.get('Nom site Urbyn'),
                    'code_prestation': row.get('Code de la prestation'),
                    'prestataire': row.get('Nom prestataire (FORMULE)')
                }

    def map_dechet(self, dechet_prest):
        if pd.isna(dechet_prest) or str(dechet_prest).strip() == '':
            return self.default_dechet_fin
        key = str(dechet_prest).strip().lower()
        return self.dechet_lookup.get(key, self.default_dechet_fin)

    def map_agrege(self, dechet_fin):
        if pd.isna(dechet_fin): return None
        return self.dechet_to_agrege.get(str(dechet_fin).strip().lower())

    def map_traitement(self, agrege, code=None):
        if pd.isna(agrege): return None, None

        if pd.notna(code) and str(code).strip():
            key = f"{agrege}{str(code).strip()}"
            if key in self.traitement_lookup:
                r = self.traitement_lookup[key]
                return r['code'], r['traitement']

        if agrege in self.traitement_lookup:
            r = self.traitement_lookup[agrege]
            return r['code'], r['traitement']

        return None, None

    def map_site(self, site_name):
        if pd.isna(site_name): return None

        key = str(site_name).strip().lower()

        # Exact match
        if key in self.site_lookup:
            return self.site_lookup[key]

        # Partial match
        for lookup_key, value in self.site_lookup.items():
            if key in lookup_key or lookup_key in key:
                return value

        return None

# =============================================================================
# MAIN FUNCTIONS
# =============================================================================
def read_input(input_file):
    print("\n" + "="*80)
    print("READING INPUT FILE (CSV)")
    print("="*80)

    if not os.path.exists(input_file):
        raise ValueError(f"File not found: {input_file}")

    # Detect separator
    sep = detect_csv_separator(input_file)
    print(f"  Detected separator: '{sep}'")

    # Detect header row
    header_row = detect_header_row(input_file, sep)
    print(f"  Header row: {header_row}")

    # Read CSV
    df = pd.read_csv(input_file, sep=sep, header=header_row, encoding='utf-8-sig')
    print(f"✓ Loaded: {len(df)} rows, {len(df.columns)} columns")

    # Detect columns
    cols = {}
    found_cols = []

    # Find site name first
    cols['site_name'] = find_column(df, COLUMN_PATTERNS['site_name'])
    if cols['site_name']:
        found_cols.append(cols['site_name'])

    # Find other columns
    for key, patterns in COLUMN_PATTERNS.items():
        if key == 'site_name':
            continue
        cols[key] = find_column(df, patterns, exclude_cols=found_cols)
        if cols[key]:
            found_cols.append(cols[key])

    # Handle duplicate column names (Code D/R appears twice)
    # Find the second Code D/R for final treatment
    code_dr_cols = [c for c in df.columns if 'code d/r' in str(c).lower()]
    if len(code_dr_cols) >= 2:
        cols['treatment_code_intermediate'] = code_dr_cols[0]
        cols['treatment_code_final'] = code_dr_cols[1]
    elif len(code_dr_cols) == 1:
        cols['treatment_code_intermediate'] = code_dr_cols[0]
        cols['treatment_code_final'] = code_dr_cols[0]

    print("\n  Detected columns:")
    for k, v in cols.items():
        print(f"    {k}: {v if v else 'NOT FOUND'}")

    return df, cols

def read_template(template_file):
    print("\n" + "="*80)
    print("READING TEMPLATE")
    print("="*80)
    df = pd.read_excel(template_file, sheet_name=0, header=TEMPLATE_HEADER_ROW)
    print(f"✓ Template: {len(df.columns)} columns")
    return df

def transform(df_input, df_template, mapper, cols):
    print("\n" + "="*80)
    print("TRANSFORMING DATA")
    print("="*80)

    output_rows = []
    skipped = 0

    for _, row in df_input.iterrows():
        # Site mapping (by NOM)
        site_name = safe_get(row, cols.get('site_name'), '')
        site_info = mapper.map_site(site_name)

        if site_info is None:
            skipped += 1
            continue

        # Waste mapping (Gisement → Déchet fin → Déchets agrégé)
        dechets_prest = safe_get(row, cols.get('waste'), '')
        dechet_fin = mapper.map_dechet(dechets_prest)
        dechets_agrege = mapper.map_agrege(dechet_fin)

        # Treatment mapping
        code_prest = safe_get(row, cols.get('treatment_code_intermediate'), '')
        code_final_from_input = safe_get(row, cols.get('treatment_code_final'), '')
        code_final, traitement = mapper.map_traitement(dechets_agrege, code_prest)

        # If no mapping found, use code from input
        if code_final is None:
            code_final = code_final_from_input

        # Weight (already in kg)
        masse_kg = safe_get(row, cols.get('weight'), 0)
        if masse_kg:
            masse_kg = float(masse_kg)

        # Date
        date_val = safe_get(row, cols.get('date'))

        # Exutoire
        exutoire = safe_get(row, cols.get('exutoire'))

        # Waste code
        waste_code = safe_get(row, cols.get('waste_code'))

        new_row = {
            'Libellé': None,
            'Groupe': GROUPE,
            'Code site': extract_code_site(site_info['nom_site']) if site_info else None,
            'Nom du site': site_info['nom_site'] if site_info else 'Site inconnu',
            'Nom du client': CLIENT,
            'Type de porteur': 'FM',
            'Commentaire mouvement': None,
            'Code de la prestation': site_info['code_prestation'] if site_info else None,
            'Prestataire': site_info['prestataire'] if site_info else GROUPE_PRESTATAIRE,
            'Groupe de Prestataire': GROUPE_PRESTATAIRE,
            'Type de prestataire': TYPE_PRESTATAIRE,
            'Périodicité': 'Jour',
            'Date début registre': date_val,
            'Date fin registre': date_val,
            'Code déchet prestataire': waste_code,
            'Déchet fin': dechet_fin,
            'Déchets agrégé': dechets_agrege,
            'Déchets prestataire': dechets_prest if dechets_prest else dechet_fin,
            'Masse totale (kg)': masse_kg,
            'Nombre de contenants': 1,
            'Volume contenant (L)': None,
            'Type de contenant': None,
            'Volume total (L)': None,
            'Nature de quantités collectées': 'Masse',
            'Qualité quantités': 'Document prestataire',
            'Précision estimations des quantités': None,
            'Traitement': traitement,
            'Traitement prestataire': None,
            'Code traitement': code_final,
            'Code traitement prestataire': code_prest,
            'Qualité du Traitement': 'Document prestataire',
            'N° de BSD/BSDD': None,
            'N° de recépissé': None,
            'Transporteur': None,
            'Transporteur prestataire': None,
            "Plaque d'immatriculation": None,
            'Exutoire intermédiaire': None,
            'Exutoire intermédiaire prestataire': None,
            "Qualité de l'exutoire intermédiaire": None,
            'Exutoire final': exutoire,
            'Exutoire final prestataire': exutoire,
            "Qualité de l'exutoire final": 'Document prestataire',
            'Période de clôture': None,
            'Statut du mouvement': 'Réalisée',
            'Commentaire': None,
        }
        output_rows.append(new_row)

    df_output = pd.DataFrame(output_rows)
    for col in df_template.columns:
        if col not in df_output.columns:
            df_output[col] = None
    df_output = df_output[df_template.columns]

    print(f"✓ Transformed {len(df_output)} rows")
    if skipped:
        print(f"  ⚠ Skipped {skipped} rows (no site mapping)")
    return df_output

def save_output(df_output, output_file):
    print("\n" + "="*80)
    print("SAVING OUTPUT")
    print("="*80)
    df_output.to_excel(output_file, sheet_name=OUTPUT_SHEET, index=False)
    print(f"✓ Saved: {output_file}")

def main():
    print("\n" + "="*80)
    print("ELISE ETL TRANSFORMATION")
    print("="*80)

    try:
        mapper = ETLMapper(ETL_FILE, PRESTATAIRE_PATTERN)
        df_input, cols = read_input(INPUT_FILE)
        df_template = read_template(TEMPLATE_FILE)
        df_output = transform(df_input, df_template, mapper, cols)
        save_output(df_output, OUTPUT_FILE)

        print("\n" + "="*80)
        print("✓✓✓ SUCCESS ✓✓✓")
        print("="*80)
    except Exception as e:
        print(f"\n✗ ERROR: {e}")
        import traceback
        traceback.print_exc()

if __name__ == "__main__":
    main()


ELISE ETL TRANSFORMATION
LOADING ETL MAPPINGS FOR: Elise
✓ Déchet: 260 mappings
✓ Paramètres: 121 mappings
✓ Traitement: 169 mappings
✓ Site: 11 mappings

READING INPUT FILE (CSV)
  Detected separator: ';'
  Header row: 1
✓ Loaded: 11 rows, 26 columns

  Detected columns:
    site_name: NOM
    site_address: N° + Rue
    site_cp: CP
    site_ville: VILLE
    date: Date Collecte
    waste: Gisement
    waste_code: Code Nomenclature Déchets (cf base article)
    weight: Quantité (kg)
    collector_site: Site
    treatment_code_intermediate: Code D/R
    treatment_code_final: Code D/R.1
    exutoire: Installation

READING TEMPLATE
✓ Template: 64 columns

TRANSFORMING DATA
✓ Transformed 11 rows

SAVING OUTPUT
✓ Saved: /content/Elise_Registre_Agrege_OUTPUT.xlsx

✓✓✓ SUCCESS ✓✓✓


#Screlec

In [ ]:
"""
Screlec ETL Transformation Script (Google Colab Version)
Transforms Screlec BSDD (Bordereau de Suivi des Déchets Dangereux) data to Urbyn format.

Key characteristics:
- Input is BSDD format (dangerous waste tracking form)
- Site identified by 'Expéditeur Nom usuel' (e.g., "147", "FR181 AIX EN PCE")
- Waste type from 'Dénomination usuelle' (e.g., "Piles en mélange")
- Weight in TONNES: 'Quantité réceptionnée nette (tonnes)'
- Treatment code: 'Code opération réalisé'
- BSD number available: 'N° de bordereau'
- Dangerous waste code: 'Code du déchet' (e.g., "20 01 33*")

Note: Screlec is an éco-organisme for batteries/piles collection.
"""

import pandas as pd
import numpy as np
import os
import re

# =============================================================================
# CONFIGURATION
# =============================================================================
INPUT_FILE = '/content/Reporting_Screlec_Capgemini_02_25.xlsx'
TEMPLATE_FILE = '/content/Modèle vierge de Registre des déchets et Reporting des coûts SLIMAN.xlsx'
ETL_FILE = '/content/ETL _ Mapping registre déchets prestataire vers Urbyn.xlsx'
OUTPUT_FILE = '/content/Screlec_Registre_Agrege_OUTPUT.xlsx'

PRESTATAIRE_PATTERN = 'Screlec'
TEMPLATE_HEADER_ROW = 8
OUTPUT_SHEET = 'Registre des déchets (Mouvement'

CLIENT = "CAPGEMINI TECHNOLOGY SERVICES"
GROUPE = "Capgemini"
GROUPE_PRESTATAIRE = "Screlec"
TYPE_PRESTATAIRE = "Eco-organisme"

# Column patterns for dynamic detection
COLUMN_PATTERNS = {
    'bsd': ['n° de bordereau', 'bordereau', 'bsd'],
    'date': ['date de réalisation', 'date d\'expédition', 'date'],
    'waste_name': ['dénomination usuelle', 'dénomination', 'déchet'],
    'waste_code': ['code du déchet', 'code déchet', 'nomenclature'],
    'weight': ['quantité réceptionnée', 'quantité acceptée', 'quantité', 'tonnes'],
    'site_name': ['expéditeur nom usuel', 'nom usuel', 'expéditeur'],
    'site_address': ['expéditeur adresse', 'adresse'],
    'treatment_code_planned': ['code opération prévu', 'opération prévu'],
    'treatment_code_done': ['code opération réalisé', 'opération réalisé'],
    'transporter': ['transporteur raison sociale', 'transporteur'],
    'destination': ['destination raison sociale', 'destination'],
    'dangerous': ['déchet dangereux', 'dangereux'],
}



# =============================================================================
# UTILITY FUNCTIONS
# =============================================================================
def find_column(df, possible_names):
    for name in possible_names:
        for col in df.columns:
            if name.lower() in str(col).lower():
                return col
    return None

def safe_get(row, column, default=None):
    if column is None:
        return default
    try:
        value = row.get(column)
        return default if pd.isna(value) else value
    except:
        return default

def extract_code_site(nom_site):
    if pd.isna(nom_site):
        return None
    nom_str = str(nom_site)
    return nom_str.split(' - ')[0] if ' - ' in nom_str else None

# =============================================================================
# ETL MAPPER CLASS
# =============================================================================
class ETLMapper:
    def __init__(self, etl_file, prestataire_pattern):
        self.etl_file = etl_file
        self.prestataire_pattern = prestataire_pattern

        print("="*80)
        print(f"LOADING ETL MAPPINGS FOR: {prestataire_pattern}")
        print("="*80)

        self.load_dechet_mapping()
        self.load_dechet_to_agrege_mapping()
        self.load_traitement_mapping()
        self.load_site_mapping()

    def load_dechet_mapping(self):
        df = pd.read_excel(self.etl_file, sheet_name='Déchet')
        mask = df['Nom prestataire (FORMULE)'].str.contains(self.prestataire_pattern, case=False, na=False)
        df_filtered = df[mask]
        print(f"✓ Déchet: {len(df_filtered)} mappings")

        self.dechet_lookup = {}
        self.default_dechet_fin = None
        for _, row in df_filtered.iterrows():
            prest = row.get('Nom des déchets prestataire')
            urbyn = row.get('Nom des déchets Urbyn')
            if pd.isna(prest):
                if pd.notna(urbyn): self.default_dechet_fin = urbyn
            elif pd.notna(urbyn):
                self.dechet_lookup[str(prest).strip().lower()] = urbyn

    def load_dechet_to_agrege_mapping(self):
        df = pd.read_excel(self.etl_file, sheet_name='Paramètres')
        self.dechet_to_agrege = {}
        for _, row in df[['Category', 'Name']].dropna().iterrows():
            self.dechet_to_agrege[str(row['Name']).strip().lower()] = str(row['Category']).strip()
        print(f"✓ Paramètres: {len(self.dechet_to_agrege)} mappings")

    def load_traitement_mapping(self):
        df = pd.read_excel(self.etl_file, sheet_name='Traitement générique')
        self.traitement_lookup = {}
        for _, row in df.iterrows():
            key = str(row.get('Concatener déchet & code de traitement prestataire', '')).strip()
            if key:
                self.traitement_lookup[key] = {
                    'code': row.get('Code traitement retraité'),
                    'traitement': row.get('Traitement')
                }
        print(f"✓ Traitement: {len(self.traitement_lookup)} mappings")

    def load_site_mapping(self):
        df = pd.read_excel(self.etl_file, sheet_name='Site')
        mask = df['Nom prestataire (FORMULE)'].str.contains(self.prestataire_pattern, case=False, na=False)
        df_filtered = df[mask]
        print(f"✓ Site: {len(df_filtered)} mappings")

        self.site_lookup = {}
        for _, row in df_filtered.iterrows():
            site_prest = row.get('Nom site prestataire')
            if pd.notna(site_prest):
                key = str(site_prest).strip().lower()
                self.site_lookup[key] = {
                    'nom_site': row.get('Nom site Urbyn'),
                    'code_prestation': row.get('Code de la prestation'),
                    'prestataire': row.get('Nom prestataire (FORMULE)')
                }

    def map_dechet(self, dechet_prest):
        if pd.isna(dechet_prest) or str(dechet_prest).strip() == '':
            return self.default_dechet_fin
        key = str(dechet_prest).strip().lower()
        return self.dechet_lookup.get(key, self.default_dechet_fin)

    def map_agrege(self, dechet_fin):
        if pd.isna(dechet_fin):
            return None
        return self.dechet_to_agrege.get(str(dechet_fin).strip().lower())

    def map_traitement(self, agrege, code=None):
        if pd.isna(agrege):
            return None, None

        if pd.notna(code) and str(code).strip():
            key = f"{agrege}{str(code).strip()}"
            if key in self.traitement_lookup:
                r = self.traitement_lookup[key]
                return r['code'], r['traitement']

        if agrege in self.traitement_lookup:
            r = self.traitement_lookup[agrege]
            return r['code'], r['traitement']

        return None, None

    def map_site(self, site_name):
        if pd.isna(site_name):
            return None

        key = str(site_name).strip().lower()

        # Exact match
        if key in self.site_lookup:
            return self.site_lookup[key]

        # Partial match
        for lookup_key, value in self.site_lookup.items():
            if key in lookup_key or lookup_key in key:
                return value

        return None

# =============================================================================
# MAIN FUNCTIONS
# =============================================================================
def read_input(input_file):
    print("\n" + "="*80)
    print("READING INPUT FILE")
    print("="*80)

    if not os.path.exists(input_file):
        raise ValueError(f"File not found: {input_file}")

    # Try to find the right sheet
    xls = pd.ExcelFile(input_file)
    sheet_name = 'registre' if 'registre' in [s.lower() for s in xls.sheet_names] else 0

    df = pd.read_excel(input_file, sheet_name=sheet_name, header=0)
    print(f"✓ Loaded: {len(df)} rows, {len(df.columns)} columns")

    # Detect columns
    cols = {k: find_column(df, v) for k, v in COLUMN_PATTERNS.items()}
    print("\n  Detected columns:")
    for k, v in cols.items():
        print(f"    {k}: {v if v else 'NOT FOUND'}")

    return df, cols

def read_template(template_file):
    print("\n" + "="*80)
    print("READING TEMPLATE")
    print("="*80)
    df = pd.read_excel(template_file, sheet_name=0, header=TEMPLATE_HEADER_ROW)
    print(f"✓ Template: {len(df.columns)} columns")
    return df

def transform(df_input, df_template, mapper, cols):
    print("\n" + "="*80)
    print("TRANSFORMING DATA")
    print("="*80)

    output_rows = []
    skipped = 0

    for _, row in df_input.iterrows():
        # Site mapping
        site_name = safe_get(row, cols.get('site_name'), '')
        site_info = mapper.map_site(site_name)

        if site_info is None:
            skipped += 1
            print(f"  ⚠ No site mapping for: '{site_name}'")
            # Still process but mark as unknown
            site_info = {'nom_site': 'Site inconnu', 'code_prestation': None}

        # Waste mapping
        dechets_prest = safe_get(row, cols.get('waste_name'), '')
        dechet_fin = mapper.map_dechet(dechets_prest)
        dechets_agrege = mapper.map_agrege(dechet_fin)

        # Treatment mapping
        code_prest = safe_get(row, cols.get('treatment_code_done'), '')
        code_final, traitement = mapper.map_traitement(dechets_agrege, code_prest)

        # If no mapping found, use code from input
        if code_final is None:
            code_final = code_prest

        # Weight conversion (TONNES → KG)
        poids_tonnes = safe_get(row, cols.get('weight'), 0)
        if poids_tonnes:
            masse_kg = float(poids_tonnes) * 1000
        else:
            masse_kg = 0

        # Date
        date_val = safe_get(row, cols.get('date'))

        # BSD number
        bsd = safe_get(row, cols.get('bsd'))

        # Waste code (dangerous)
        waste_code = safe_get(row, cols.get('waste_code'))

        # Transporter
        transporter = safe_get(row, cols.get('transporter'))

        # Destination (exutoire)
        exutoire = safe_get(row, cols.get('destination'))

        new_row = {
            'Libellé': None,
            'Groupe': GROUPE,
            'Code site': extract_code_site(site_info['nom_site']) if site_info else None,
            'Nom du site': site_info['nom_site'] if site_info else 'Site inconnu',
            'Nom du client': CLIENT,
            'Type de porteur': 'FM',
            'Commentaire mouvement': None,
            'Code de la prestation': site_info.get('code_prestation') if site_info else None,
            'Prestataire': GROUPE_PRESTATAIRE,
            'Groupe de Prestataire': GROUPE_PRESTATAIRE,
            'Type de prestataire': TYPE_PRESTATAIRE,
            'Périodicité': 'Jour',
            'Date début registre': date_val,
            'Date fin registre': date_val,
            'Code déchet prestataire': waste_code,
            'Déchet fin': dechet_fin,
            'Déchets agrégé': dechets_agrege,
            'Déchets prestataire': dechets_prest if dechets_prest else dechet_fin,
            'Masse totale (kg)': masse_kg,
            'Nombre de contenants': 1,
            'Volume contenant (L)': None,
            'Type de contenant': None,
            'Volume total (L)': None,
            'Nature de quantités collectées': 'Masse',
            'Qualité quantités': 'Document prestataire',
            'Précision estimations des quantités': None,
            'Traitement': traitement,
            'Traitement prestataire': None,
            'Code traitement': code_final,
            'Code traitement prestataire': code_prest,
            'Qualité du Traitement': 'Document prestataire',
            'N° de BSD/BSDD': bsd,
            'N° de recépissé': None,
            'Transporteur': transporter,
            'Transporteur prestataire': transporter,
            "Plaque d'immatriculation": None,
            'Exutoire intermédiaire': None,
            'Exutoire intermédiaire prestataire': None,
            "Qualité de l'exutoire intermédiaire": None,
            'Exutoire final': exutoire,
            'Exutoire final prestataire': exutoire,
            "Qualité de l'exutoire final": 'Document prestataire',
            'Période de clôture': None,
            'Statut du mouvement': 'Réalisée',
            'Commentaire': None,
        }
        output_rows.append(new_row)

    df_output = pd.DataFrame(output_rows)
    for col in df_template.columns:
        if col not in df_output.columns:
            df_output[col] = None
    df_output = df_output[df_template.columns]

    print(f"✓ Transformed {len(df_output)} rows")
    if skipped:
        print(f"  ⚠ {skipped} rows had no site mapping (marked as 'Site inconnu')")
    return df_output

def save_output(df_output, output_file):
    print("\n" + "="*80)
    print("SAVING OUTPUT")
    print("="*80)
    df_output.to_excel(output_file, sheet_name=OUTPUT_SHEET, index=False)
    print(f"✓ Saved: {output_file}")

def main():
    print("\n" + "="*80)
    print("SCRELEC ETL TRANSFORMATION")
    print("="*80)

    try:
        mapper = ETLMapper(ETL_FILE, PRESTATAIRE_PATTERN)
        df_input, cols = read_input(INPUT_FILE)
        df_template = read_template(TEMPLATE_FILE)
        df_output = transform(df_input, df_template, mapper, cols)
        save_output(df_output, OUTPUT_FILE)

        print("\n" + "="*80)
        print("✓✓✓ SUCCESS ✓✓✓")
        print("="*80)
    except Exception as e:
        print(f"\n✗ ERROR: {e}")
        import traceback
        traceback.print_exc()

if __name__ == "__main__":
    main()


SCRELEC ETL TRANSFORMATION
LOADING ETL MAPPINGS FOR: Screlec
✓ Déchet: 0 mappings
✓ Paramètres: 121 mappings
✓ Traitement: 169 mappings
✓ Site: 0 mappings

READING INPUT FILE
✓ Loaded: 3 rows, 140 columns

  Detected columns:
    bsd: N° de bordereau
    date: Date de réalisation de l'opération
    waste_name: Dénomination usuelle
    waste_code: Code du déchet
    weight: Quantité réceptionnée nette (tonnes)
    site_name: Expéditeur Nom usuel
    site_address: Expéditeur Adresse
    treatment_code_planned: Code opération prévu
    treatment_code_done: Code opération réalisé
    transporter: Transporteur raison sociale
    destination: Destination raison sociale
    dangerous: Déchet dangereux

READING TEMPLATE
✓ Template: 64 columns

TRANSFORMING DATA
  ⚠ No site mapping for: '147'
  ⚠ No site mapping for: 'FR181 AIX EN PCE'
  ⚠ No site mapping for: '147'
✓ Transformed 3 rows
  ⚠ 3 rows had no site mapping (marked as 'Site inconnu')

SAVING OUTPUT
✓ Saved: /content/Screlec_Registre_

#trackdechet

In [ ]:
"""
Trackdechet ETL Transformation Script (Google Colab Version)
Transforms Trackdechet BSDD/BSDASRI exports to Urbyn aggregated format.

Trackdechet is the French government platform for tracking hazardous waste (BSD).
This script handles exports from the platform which contain BSD data from various sources.

Key characteristics:
- Input is BSDD/BSDASRI format (142 columns)
- Site identified by 'Expéditeur Nom usuel' (e.g., "147", "AXEO", "LYON IVOIRE - FR178")
- Waste type from 'Dénomination usuelle' (e.g., "DEEE en mélange Vrac", "Piles et batteries")
- Weight in TONNES: 'Quantité réceptionnée nette (tonnes)'
- Treatment code: 'Code opération réalisé'
- BSD number: 'N° de bordereau'
- Dangerous waste code: 'Code du déchet' (e.g., "16 02 13*", "20 01 33*")
"""

import pandas as pd
import numpy as np
import os

# =============================================================================
# CONFIGURATION
# =============================================================================
INPUT_FILE = '/content/TD-Registre-20250425-Sortant.xlsx'
TEMPLATE_FILE = '/content/Modèle vierge de Registre des déchets et Reporting des coûts SLIMAN.xlsx'
ETL_FILE = '/content/ETL _ Mapping registre déchets prestataire vers Urbyn.xlsx'
OUTPUT_FILE = '/content/Trackdechet_Registre_Agrege_OUTPUT.xlsx'

PRESTATAIRE_PATTERN = 'Trackdechet'
TEMPLATE_HEADER_ROW = 8
OUTPUT_SHEET = 'Registre des déchets (Mouvement'

CLIENT = "CAPGEMINI TECHNOLOGY SERVICES"
GROUPE = "Capgemini"
GROUPE_PRESTATAIRE = "Trackdechet"
TYPE_PRESTATAIRE = "Plateforme gouvernementale"

# Column patterns for dynamic detection
COLUMN_PATTERNS = {
    'bsd': ['n° de bordereau', 'bordereau', 'bsd'],
    'date': ['date de réalisation', 'date d\'expédition', 'date'],
    'waste_name': ['dénomination usuelle', 'dénomination', 'déchet'],
    'waste_code': ['code du déchet', 'code déchet'],
    'weight': ['quantité réceptionnée', 'quantité acceptée', 'quantité'],
    'site_name': ['expéditeur nom usuel', 'nom usuel'],
    'site_address': ['expéditeur adresse', 'adresse'],
    'site_raison_sociale': ['expéditeur raison sociale'],
    'treatment_code_planned': ['code opération prévu', 'opération prévu'],
    'treatment_code_done': ['code opération réalisé', 'opération réalisé'],
    'transporter': ['transporteur raison sociale', 'transporteur'],
    'destination': ['destination raison sociale', 'destination'],
    'type_bordereau': ['type de bordereau'],
    'dangerous': ['déchet dangereux', 'dangereux'],
}

# =============================================================================
# UTILITY FUNCTIONS
# =============================================================================
def find_column(df, possible_names):
    for name in possible_names:
        for col in df.columns:
            if name.lower() in str(col).lower():
                return col
    return None

def safe_get(row, column, default=None):
    if column is None:
        return default
    try:
        value = row.get(column)
        return default if pd.isna(value) else value
    except:
        return default

def extract_code_site(nom_site):
    if pd.isna(nom_site):
        return None
    nom_str = str(nom_site)
    return nom_str.split(' - ')[0] if ' - ' in nom_str else None

# =============================================================================
# ETL MAPPER CLASS
# =============================================================================
class ETLMapper:
    def __init__(self, etl_file, prestataire_pattern):
        self.etl_file = etl_file
        self.prestataire_pattern = prestataire_pattern

        print("="*80)
        print(f"LOADING ETL MAPPINGS FOR: {prestataire_pattern}")
        print("="*80)

        self.load_dechet_mapping()
        self.load_dechet_to_agrege_mapping()
        self.load_traitement_mapping()
        self.load_site_mapping()

    def load_dechet_mapping(self):
        df = pd.read_excel(self.etl_file, sheet_name='Déchet')
        mask = df['Nom prestataire (FORMULE)'].str.contains(self.prestataire_pattern, case=False, na=False)
        df_filtered = df[mask]
        print(f"✓ Déchet: {len(df_filtered)} mappings")

        self.dechet_lookup = {}
        self.default_dechet_fin = None
        for _, row in df_filtered.iterrows():
            prest = row.get('Nom des déchets prestataire')
            urbyn = row.get('Nom des déchets Urbyn')
            if pd.isna(prest):
                if pd.notna(urbyn): self.default_dechet_fin = urbyn
            elif pd.notna(urbyn):
                self.dechet_lookup[str(prest).strip().lower()] = urbyn

    def load_dechet_to_agrege_mapping(self):
        df = pd.read_excel(self.etl_file, sheet_name='Paramètres')
        self.dechet_to_agrege = {}
        for _, row in df[['Category', 'Name']].dropna().iterrows():
            self.dechet_to_agrege[str(row['Name']).strip().lower()] = str(row['Category']).strip()
        print(f"✓ Paramètres: {len(self.dechet_to_agrege)} mappings")

    def load_traitement_mapping(self):
        df = pd.read_excel(self.etl_file, sheet_name='Traitement générique')
        self.traitement_lookup = {}
        for _, row in df.iterrows():
            key = str(row.get('Concatener déchet & code de traitement prestataire', '')).strip()
            if key:
                self.traitement_lookup[key] = {
                    'code': row.get('Code traitement retraité'),
                    'traitement': row.get('Traitement')
                }
        print(f"✓ Traitement: {len(self.traitement_lookup)} mappings")

    def load_site_mapping(self):
        df = pd.read_excel(self.etl_file, sheet_name='Site')
        mask = df['Nom prestataire (FORMULE)'].str.contains(self.prestataire_pattern, case=False, na=False)
        df_filtered = df[mask]
        print(f"✓ Site: {len(df_filtered)} mappings")

        self.site_lookup = {}
        for _, row in df_filtered.iterrows():
            site_prest = row.get('Nom site prestataire')
            if pd.notna(site_prest):
                key = str(site_prest).strip().lower()
                self.site_lookup[key] = {
                    'nom_site': row.get('Nom site Urbyn'),
                    'code_prestation': row.get('Code de la prestation'),
                    'prestataire': row.get('Nom prestataire (FORMULE)')
                }

    def map_dechet(self, dechet_prest):
        if pd.isna(dechet_prest) or str(dechet_prest).strip() == '':
            return self.default_dechet_fin
        key = str(dechet_prest).strip().lower()
        return self.dechet_lookup.get(key, self.default_dechet_fin)

    def map_agrege(self, dechet_fin):
        if pd.isna(dechet_fin):
            return None
        return self.dechet_to_agrege.get(str(dechet_fin).strip().lower())

    def map_traitement(self, agrege, code=None):
        if pd.isna(agrege):
            return None, None

        if pd.notna(code) and str(code).strip():
            key = f"{agrege}{str(code).strip()}"
            if key in self.traitement_lookup:
                r = self.traitement_lookup[key]
                return r['code'], r['traitement']

        if agrege in self.traitement_lookup:
            r = self.traitement_lookup[agrege]
            return r['code'], r['traitement']

        return None, None

    def map_site(self, site_name):
        if pd.isna(site_name):
            return None

        key = str(site_name).strip().lower()

        # Exact match
        if key in self.site_lookup:
            return self.site_lookup[key]

        # Partial match
        for lookup_key, value in self.site_lookup.items():
            if key in lookup_key or lookup_key in key:
                return value

        return None

# =============================================================================
# MAIN FUNCTIONS
# =============================================================================
def read_input(input_file):
    print("\n" + "="*80)
    print("READING INPUT FILE")
    print("="*80)

    if not os.path.exists(input_file):
        raise ValueError(f"File not found: {input_file}")

    # Try to find the right sheet
    xls = pd.ExcelFile(input_file)
    sheet_name = 'registre' if 'registre' in [s.lower() for s in xls.sheet_names] else 0

    df = pd.read_excel(input_file, sheet_name=sheet_name, header=0)
    print(f"✓ Loaded: {len(df)} rows, {len(df.columns)} columns")

    # Detect columns
    cols = {k: find_column(df, v) for k, v in COLUMN_PATTERNS.items()}
    print("\n  Detected columns:")
    for k, v in cols.items():
        print(f"    {k}: {v if v else 'NOT FOUND'}")

    return df, cols

def read_template(template_file):
    print("\n" + "="*80)
    print("READING TEMPLATE")
    print("="*80)
    df = pd.read_excel(template_file, sheet_name=0, header=TEMPLATE_HEADER_ROW)
    print(f"✓ Template: {len(df.columns)} columns")
    return df

def transform(df_input, df_template, mapper, cols):
    print("\n" + "="*80)
    print("TRANSFORMING DATA")
    print("="*80)

    output_rows = []
    skipped_no_site = 0
    skipped_no_weight = 0

    for _, row in df_input.iterrows():
        # Skip rows without weight (incomplete BSD)
        weight = safe_get(row, cols.get('weight'))
        if pd.isna(weight) or weight == 0:
            skipped_no_weight += 1
            continue

        # Site mapping
        site_name = safe_get(row, cols.get('site_name'), '')
        site_info = mapper.map_site(site_name)

        if site_info is None:
            skipped_no_site += 1
            print(f"  ⚠ No site mapping for: '{site_name}'")
            # Still process but mark as unknown
            site_info = {'nom_site': 'Site inconnu', 'code_prestation': None, 'prestataire': None}

        # Waste mapping
        dechets_prest = safe_get(row, cols.get('waste_name'), '')
        dechet_fin = mapper.map_dechet(dechets_prest)
        dechets_agrege = mapper.map_agrege(dechet_fin)

        # Treatment mapping
        code_prest = safe_get(row, cols.get('treatment_code_done'), '')
        code_final, traitement = mapper.map_traitement(dechets_agrege, code_prest)

        # If no mapping found, use code from input
        if code_final is None:
            code_final = code_prest

        # Weight conversion (TONNES → KG)
        masse_kg = float(weight) * 1000 if weight else 0

        # Date - prefer date de réalisation, fallback to date d'expédition
        date_val = safe_get(row, cols.get('date'))

        # BSD number
        bsd = safe_get(row, cols.get('bsd'))

        # Waste code (dangerous)
        waste_code = safe_get(row, cols.get('waste_code'))

        # Transporter
        transporter = safe_get(row, cols.get('transporter'))

        # Destination (exutoire)
        exutoire = safe_get(row, cols.get('destination'))

        # Prestataire from transporter (since Trackdechet is a platform, not a prestataire)
        prestataire = transporter if transporter else GROUPE_PRESTATAIRE

        new_row = {
            'Libellé': None,
            'Groupe': GROUPE,
            'Code site': extract_code_site(site_info['nom_site']) if site_info else None,
            'Nom du site': site_info['nom_site'] if site_info else 'Site inconnu',
            'Nom du client': CLIENT,
            'Type de porteur': 'FM',
            'Commentaire mouvement': None,
            'Code de la prestation': site_info.get('code_prestation') if site_info else None,
            'Prestataire': prestataire,
            'Groupe de Prestataire': GROUPE_PRESTATAIRE,
            'Type de prestataire': TYPE_PRESTATAIRE,
            'Périodicité': 'Jour',
            'Date début registre': date_val,
            'Date fin registre': date_val,
            'Code déchet prestataire': waste_code,
            'Déchet fin': dechet_fin,
            'Déchets agrégé': dechets_agrege,
            'Déchets prestataire': dechets_prest if dechets_prest else dechet_fin,
            'Masse totale (kg)': masse_kg,
            'Nombre de contenants': 1,
            'Volume contenant (L)': None,
            'Type de contenant': None,
            'Volume total (L)': None,
            'Nature de quantités collectées': 'Masse',
            'Qualité quantités': 'Document prestataire',
            'Précision estimations des quantités': None,
            'Traitement': traitement,
            'Traitement prestataire': None,
            'Code traitement': code_final,
            'Code traitement prestataire': code_prest,
            'Qualité du Traitement': 'Document prestataire',
            'N° de BSD/BSDD': bsd,
            'N° de recépissé': None,
            'Transporteur': transporter,
            'Transporteur prestataire': transporter,
            "Plaque d'immatriculation": None,
            'Exutoire intermédiaire': None,
            'Exutoire intermédiaire prestataire': None,
            "Qualité de l'exutoire intermédiaire": None,
            'Exutoire final': exutoire,
            'Exutoire final prestataire': exutoire,
            "Qualité de l'exutoire final": 'Document prestataire',
            'Période de clôture': None,
            'Statut du mouvement': 'Réalisée',
            'Commentaire': None,
        }
        output_rows.append(new_row)

    df_output = pd.DataFrame(output_rows)
    for col in df_template.columns:
        if col not in df_output.columns:
            df_output[col] = None
    df_output = df_output[df_template.columns]

    print(f"✓ Transformed {len(df_output)} rows")
    if skipped_no_weight:
        print(f"  ⚠ Skipped {skipped_no_weight} rows (no weight/incomplete BSD)")
    if skipped_no_site:
        print(f"  ⚠ {skipped_no_site} rows had no site mapping (marked as 'Site inconnu')")
    return df_output

def save_output(df_output, output_file):
    print("\n" + "="*80)
    print("SAVING OUTPUT")
    print("="*80)
    df_output.to_excel(output_file, sheet_name=OUTPUT_SHEET, index=False)
    print(f"✓ Saved: {output_file}")

def main():
    print("\n" + "="*80)
    print("TRACKDECHET ETL TRANSFORMATION")
    print("="*80)

    try:
        mapper = ETLMapper(ETL_FILE, PRESTATAIRE_PATTERN)
        df_input, cols = read_input(INPUT_FILE)
        df_template = read_template(TEMPLATE_FILE)
        df_output = transform(df_input, df_template, mapper, cols)
        save_output(df_output, OUTPUT_FILE)

        print("\n" + "="*80)
        print("✓✓✓ SUCCESS ✓✓✓")
        print("="*80)
    except Exception as e:
        print(f"\n✗ ERROR: {e}")
        import traceback
        traceback.print_exc()

if __name__ == "__main__":
    main()


TRACKDECHET ETL TRANSFORMATION
LOADING ETL MAPPINGS FOR: Trackdechet
✓ Déchet: 0 mappings
✓ Paramètres: 121 mappings
✓ Traitement: 169 mappings
✓ Site: 0 mappings

READING INPUT FILE
✓ Loaded: 9 rows, 142 columns

  Detected columns:
    bsd: N° de bordereau
    date: Date de réalisation de l'opération
    waste_name: Dénomination usuelle
    waste_code: Code du déchet
    weight: Quantité réceptionnée nette (tonnes)
    site_name: Expéditeur Nom usuel
    site_address: Expéditeur Adresse
    site_raison_sociale: Expéditeur raison sociale
    treatment_code_planned: Code opération prévu
    treatment_code_done: Code opération réalisé
    transporter: Transporteur raison sociale
    destination: Destination raison sociale
    type_bordereau: Type de bordereau
    dangerous: Déchet dangereux

READING TEMPLATE
✓ Template: 64 columns

TRANSFORMING DATA
  ⚠ No site mapping for: '147'
  ⚠ No site mapping for: 'LYON IVOIRE - FR178'
  ⚠ No site mapping for: 'LYON IVOIRE - FR178'
  ⚠ No site m